In [83]:
##### LIBRARIES #####
import pickle
import json
import numpy as np
import pandas as pd
import random
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
from gurobipy import *
from gurobipy import GRB
import gurobipy as gp
from itertools import product
import time
import csv
import itertools
import os
import sys
import pathlib
import math
from datetime import datetime
import shutil
from statistics import mean
import threading
from sklearn.model_selection import train_test_split
from random import shuffle
from mnist import MNIST # import mnist parser
import itertools
import time
import logging
from sklearn.model_selection import KFold # for k folding
from sklearn.model_selection import RepeatedKFold # for k folding
from sklearn.preprocessing import Binarizer
from sklearn.preprocessing import LabelBinarizer
from sklearn.preprocessing import OneHotEncoder

In [84]:
# config pc
global pc_name
global log_folder
global pushover_token
global output_folder

if os.environ['COMPUTERNAME'] == "KTMELDCGLXJQ3":
    log_folder = 'logs/elderax/'
    output_folder = 'outputs/elderax/'
    pc_name = 'Elderax'
    pushover_token = "aqmbyd1nsw7ir86sdx1mchq9o68vpg"
    
elif os.environ['COMPUTERNAME'] == "TSMELDPC0YSGRF":
    log_folder = 'logs/griffon/'
    output_folder = 'outputs/griffon/'
    pc_name = 'Griffon'
    pushover_token = "aqmbyd1nsw7ir86sdx1mchq9o68vpg"

elif os.environ['COMPUTERNAME'] == "KTMELLB8HRYZ2":
    log_folder = 'logs/henry/'
    output_folder = 'outputs/henry/'
    pc_name = 'Henry'
    pushover_token = "axa4j4ryxzj5uvt53hv67scaunihs9"

elif os.environ['COMPUTERNAME'] == "QUEEN-DESKTOP":
    log_folder = 'logs/queen/'
    output_folder = 'outputs/queen/'
    pc_name = 'Queen'
    pushover_token = "axa4j4ryxzj5uvt53hv67scaunihs9"
    
else:
    log_folder = 'logs/unknown/'
    pc_name = 'unknown'

In [85]:
def create_record_folder(dataset, ibudget, start_time_main):
    folder_path = f"records/{pc_name}/{start_time_main.strftime('%Y-%m-%d_%H-%M-%S')}_dataset-{dataset}_ibudget-{ibudget}/" # folder name
    try:
        os.mkdir(folder_path) # create folder
        return folder_path # return folder name
    except OSError as e:
        print(e)
        return folder_path # return folder name

In [86]:
def dict_save(saving_dict, file_name, ext, folder=output_folder):
    
    i = 10000
    while os.path.exists(f"{folder}{file_name}-{i}.{ext}"):
        i += 1

    with open(f"{folder}{file_name}-{i}.{ext}", 'wb') as f:
        pickle.dump(saving_dict, f)
        
def dict_load(file_name, pc_name=pc_name):

    with open('outputs/' + pc_name.lower() + '/' + file_name, 'rb') as f:
        loading_dict = pickle.load(f)
    return dict(loading_dict)
    

In [87]:
# config loguru
from loguru import logger
    
logger.remove() # clear existing loggers/sinks

# adv example
def log_adv_example_gen(msg):
    logger_adv = logger.bind(task="adv_example_gen")
    logger_adv.info(msg)

def clear_log_adv_example_gen():
    open(log_folder + "adv_example_gen.log", 'w').close()

logger.add(log_folder + "adv_example_gen.log",
           format="{time:HH:mm:ss.SS} | {level} | {message}",
           filter=lambda record: record["extra"]["task"] == "adv_example_gen",)

# adv train
def log_adv_train(msg):
    logger_adv_train = logger.bind(task="adv_train")
    logger_adv_train.info(msg)

def clear_log_adv_train():
    open(log_folder + "adv_train.log", 'w').close()

logger.add(log_folder + "adv_train.log",
           format="{time:HH:mm:ss.SS} | {level} | {message}",
           filter=lambda record: record["extra"]["task"] == "adv_train",)

# optimize model
def log_model_optimize(msg):
    logger_model_optimize = logger.bind(task="model_optimize")
    logger_model_optimize.info(msg)

def clear_log_model_optimize():
    open(log_folder + "model_optimize.log", 'w').close()

logger.add(log_folder + "model_optimize.log",
           format="{time:HH:mm:ss.SS} | {level} | {message}",
           filter=lambda record: record["extra"]["task"] == "model_optimize",)

# cut vs2
def log_adv_cut_v2(msg):
    logger_model_optimize = logger.bind(task="adv_cut_v2")
    logger_model_optimize.info(msg)

def clear_log_adv_cut_v2():
    open(log_folder + "adv_cut_v2.log", 'w').close()

logger.add(log_folder + "adv_cut_v2.log",
           format="{time:HH:mm:ss.SS} | {level} | {message}",
           filter=lambda record: record["extra"]["task"] == "adv_cut_v2",)

# main
def log_main(msg):
    logger_main = logger.bind(task="main")
    logger_main.info(msg)

def clear_log_main():
    open(log_folder + "main.log", 'w').close()

logger.add(log_folder + "main.log",
           format="{time:HH:mm:ss.SS} | {level} | {message}",
           filter=lambda record: record["extra"]["task"] == "main",)

15

In [88]:
# config onedrive
def onedrive(action='start', active=0):
    # stop your onedrive (else will keep syncing logs)   
    
    # on by default on Griffon and Henry
    if pc_name == 'Griffon' or pc_name == 'Henry' or pc_name == 'Queen' or active == 1:

        if action == 'start':    
            if pc_name == "Elderax":
                !start "OneDrive" /B "%LOCALAPPDATA%\Microsoft\OneDrive\onedrive" /background
                return None
            if pc_name == "Griffon":
                !start "OneDrive" /B "C:\Program Files\Microsoft OneDrive\onedrive" /background
                return None
            if pc_name == "Henry":
                !start "OneDrive" /B "%LOCALAPPDATA%\Microsoft\OneDrive\onedrive" /background
                return None
            if pc_name == "Queen":
                !start "OneDrive" /B "C:\Program Files\Microsoft OneDrive\onedrive" /background
                return None
        
        if action == 'stop':
            if pc_name == "Elderax":
                !"%LOCALAPPDATA%\Microsoft\OneDrive\onedrive" /shutdown
                return None
            if pc_name == "Griffon":
                !"C:\Program Files\Microsoft OneDrive\onedrive" /shutdown
                return None
            if pc_name == "Henry":
                !"%LOCALAPPDATA%\Microsoft\OneDrive\onedrive" /shutdown
                return None
            if pc_name == "Queen":
                !"C:\Program Files\Microsoft OneDrive\onedrive" /shutdown
                return None

In [89]:
# pushover config
import http.client, urllib
def pushover(title="Model", msg="Test", active=0, device="Elderax_Edge,Elderax_Chrome,Y8"):
    
    # on by default on Griffon and Henry
    if pc_name == 'Griffon' or pc_name == 'Henry' or pc_name == 'Queen' or active == 1:
        
        conn = http.client.HTTPSConnection("api.pushover.net:443")
        conn.request("POST", "/1/messages.json",
        urllib.parse.urlencode({
            "token": pushover_token,
            "user": "BAguidPakA2RfKFaNhqQfCsrbExyce",
            "message": msg,
            "title": title,
            "device": pc_name,
        }), { "Content-type": "application/x-www-form-urlencoded" })
        conn.getresponse()

In [90]:
##### DATA #####

In [91]:
def data_binazrize(row, col_means):
    """ binarise, 0 if <= avg, 1 if > avg """
    
    for c1 in row.index: # for each col in row
        for c2 in col_means.index: # for each col in means
            if c1==c2: # if col names match do a comparison
                if row[c1] > col_means[c1]: # if row value greater then mean 1
                    row[c1] = 1
                else:
                    row[c1] = 0
        # print(r)
    return row

In [92]:
def data_load_mnist(f=0, n=0):
    # mndata = MNIST('.data/mnist/')
    mn_path = os.path.join( os.getcwd(), 'data\\mnist\\') # combining with working directory
    mndata = MNIST(mn_path)
    images, labels = mndata.load_training()
        
    dfx = pd.DataFrame(images)
    dfy = pd.DataFrame(labels)
    dfy = pd.get_dummies(dfy[0]) # one hot encode

    # reduce to f features if f != 0 (for loading less data)
    if f > 0:
        dfx = dfx.loc[:, :f]
        dfy = dfy.loc[:, :f]
    # reduce to n samples if n != 0 (for loading less data)
    if n > 0:
        dfx = dfx.loc[:n, :]
        dfy = dfy.loc[:n, :]
    
    # shuffle datasets in sync
    idx = np.random.permutation(dfy.index)
    dfx = dfx.reindex(idx).reset_index(drop=True)
    dfy = dfy.reindex(idx).reset_index(drop=True)
    
    # split test and training set
    dfx_train, dfx_test, dfy_train, dfy_test = train_test_split(dfx, dfy, test_size=0.3, random_state=11)
    
    # reset indexes
    dfx_train = dfx_train.reset_index(drop=True)
    dfx_test = dfx_test.reset_index(drop=True)
    dfy_train = dfy_train.reset_index(drop=True)
    dfy_test = dfy_test.reset_index(drop=True) 
    
    # print(f'dfx_train: {dfx_train.shape} | dfy_train: {dfy_train.shape} || dfx_test: {dfx_test.shape} | dfy_test:{dfy_test}')

    return dfx_train, dfx_test, dfy_train, dfy_test


In [93]:
def data_prepare(file_path, header, col_y_idx0, dfx_col_idx0_drop_ls, 
                 bin_cols_dict, delimiter, print_out, test_size=0.3, k=0, n=0, seed_val=123):
    """ prepare data for use by load_data() """
    
    # file_path = './data/zoo/zoo.data'
    # header = None # None or 0 (first row)
    # col_y_idx0 = 17 # col num (ind 0) that has predicted class from df
    # dfx_col_idx0_drop_ls = [col_y_idx0, 0] # y column and any other idx if need be (0 here)
    # bin_cols_dict = {} # colname, bins

    # load data
    df = pd.read_csv(file_path, header=header, delimiter=delimiter)

    ######################## DF #########################
    # print(df)

    # shuffle data
    
    np.random.seed(seed_val) # goes down by 1
    
    # np.random.seed(1) # goes down by 1
    # np.random.seed(12) # same
    # np.random.seed(123) # 1 and solved = robust
    # np.random.seed(1234) # same
    # np.random.seed(12345) # up 1 data point
    # np.random.seed(123456) # 3 down to robustness
    # np.random.seed(1234567) # reduce 1 example
    # np.random.seed(12345678) # 1 becomes robust
    # df = df.sample(frac=1).reset_index(drop=True)
    
    # limit to n if given (0 mean all datapoints)
    if n != 0:
        df = df.sample(n=n).reset_index(drop=True)
    else:
        df = df.sample(frac=1).reset_index(drop=True)
    ######################## DFY #########################

    # get predicted class
    dfy = pd.DataFrame(df.iloc[: , col_y_idx0]) # get the y labels
    dfy.columns = range(dfy.columns.size) # reset column name

    # factorise y (no harm in it)
    dfy = dfy.apply(lambda x: pd.factorize(x, sort=True)[0]) # factorise all columns in dfy

    # one hot encode
    dfy = OneHotEncoder().fit_transform(dfy).toarray()
    dfy = pd.DataFrame(dfy).astype(int)

    ######################## DFX #########################

    # prepare
    dfx = df.drop(df.columns[dfx_col_idx0_drop_ls], axis=1) # drop columns
    dfx.columns = range(dfx.columns.size) # reset column name

    # code continous variables (make sure to check idx after dropping cols)
    if len(bin_cols_dict) > 0:
        for colname, bins in bin_cols_dict.items():
            array_temp1 = pd.cut(df[colname], bins) # split to 5
            dfx[colname] = pd.factorize(array_temp1, sort=True)[0]
            # print(dfx[colname])

    # get no. of responses per feature
    x_uniq_dict = {}
    for colname in dfx:
        x_uniq_dict[colname] = dfx[colname].nunique()

    dfx_out = pd.DataFrame() # build output dataframe

    # factorise (just incase + resets any other encoding)
    for colname, uniq in x_uniq_dict.items():
        
        if uniq <= 2: # for those with 2 classes (or all 1/0)
            array_temp2 = pd.factorize(dfx[colname], sort=True)[0] # factorise in case and add to dfx_out
        elif uniq > 2: # if more then 2 classes
            array_temp2 = OneHotEncoder().fit_transform(dfx[[colname]]).toarray()
        
        dfx_temp = pd.DataFrame(array_temp2).astype(int) # convert to df and make int
            
        new_df = len([x for x in dfx_out]) == 0 # if dfx_out is blank ie no columns
        if new_df:
            dfx_out = dfx_temp
            if print_out >= 2:
                print(f"col: {colname} --> onehot --> {uniq} cols")
        else:
            dfx_out = dfx_out.join(dfx_temp, rsuffix="_o") # merge into out df
            dfx_out.columns = range(dfx_out.columns.size) # reset column names
            if print_out >= 2:
                print(f"col: {colname} --> onehot --> {uniq} cols")

    dfx = dfx_out # set dfx to dfx_out (for consistancy)
    
    # split test and training (test/train or k-fold)
    
    # k val
    if k == 0:
        returned_dfs = train_test_split(dfx, dfy, test_size=test_size, random_state=123456)
        (dfx_train, dfx_test, dfy_train, dfy_test) = returned_dfs
        # print(df) # checking
        # print(dfy) # checking
        # print(dfx) # checking

        # reset indexes
        dfx_train = dfx_train.reset_index(drop=True)
        dfx_test = dfx_test.reset_index(drop=True)
        dfy_train = dfy_train.reset_index(drop=True)
        dfy_test = dfy_test.reset_index(drop=True)
    
        if print_out == 1:
            print(f'train/test split w/test_size: {test_size}')
            print(f'y_classes: {dfy_train.shape[1]} | x_features: {dfx_train.shape[1]} | n-train: {dfx_train.shape[0]} | n-test: {dfx_test.shape[0]}')
         
    else:
        rkf = RepeatedKFold(n_splits=k, n_repeats=1, random_state=123456)
        # rkf = KFold(n_splits=4, random_state=None)

        dfx_train_ls = []
        dfx_test_ls = []
        
        dfy_train_ls = []
        dfy_test_ls = []
 
        for train_index, test_index in rkf.split(dfx):
            # print(train_index, test_index)
            x_train = dfx.loc[train_index].reset_index(drop=True)
            x_test = dfx.loc[test_index].reset_index(drop=True)
            dfx_train_ls.append(x_train)
            dfx_test_ls.append(x_test)
            
            y_train = dfy.loc[train_index].reset_index(drop=True)
            y_test = dfy.loc[test_index].reset_index(drop=True)
            dfy_train_ls.append(y_train)
            dfy_test_ls.append(y_test)

        print(f'k-fold split w/k folds: {k}')
        print(f'Per Split || y_classes: {dfy_train_ls[0].shape[1]} | x_features: {dfx_train_ls[0].shape[1]} | n-train: {dfx_train_ls[0].shape[0]} | n-test: {dfx_test_ls[0].shape[0]}')
    # print shape

    if k == 0: # return original + train/test dfs
        return df, dfx_train, dfx_test, dfy_train, dfy_test
    else: # return original + tarin/test df lists
        return df, dfx_train_ls, dfx_test_ls, dfy_train_ls, dfy_test_ls


In [94]:
def data_load(dataset, print_out=0, test_size=0.3, k=0, n=0, seed_val=123):
    print_out = print_out # print prep details
    
    if dataset=='zoo':
        file_path = './data/zoo/zoo.data'
        header = None # None or 0 (first row)
        delimiter = None # None = ',' or pass str if something
        col_y_idx0 = 17 # col num (ind 0) that has predicted class from df
        dfx_col_idx0_drop_ls = [col_y_idx0, 0] # y column and any other idx if need be
        bin_cols_dict = {} # colname, bins
    
    if dataset=='student':
        file_path = './data/student/DATA.csv'
        header = 0 # None or 0 (first row)
        delimiter = ';' # None = ',' or pass str if something
        col_y_idx0 = 32 # col num (ind 0) that has predicted class from df
        dfx_col_idx0_drop_ls = [col_y_idx0, 0] # y column and any other idx if need be
        bin_cols_dict = {} # colname, bins

    if dataset=='phishing':
        file_path = './data/phishing/PhishingData.csv'
        header = None # None or 0 (first row)
        delimiter = None # None = ',' or pass str if something
        col_y_idx0 = 9 # col num (ind 0) that has predicted class from df
        dfx_col_idx0_drop_ls = [col_y_idx0] # y column and any other idx if need be
        bin_cols_dict = {} # colname, bins
        
    if dataset=='heart':
        file_path = './data/heart/processed.cleveland.data'
        header = None # None or 0 (first row)
        delimiter = None # None = ',' or pass str if something
        col_y_idx0 = 13 # col num (ind 0) that has predicted class from df
        dfx_col_idx0_drop_ls = [col_y_idx0] # y column and any other idx if need be
        bin_cols_dict = {3:20, 4:20, 7:20, 9:20} # colname, bins

    if dataset=='wine':
        file_path = './data/wine/wine.data.csv'
        header = None # None or 0 (first row)
        delimiter = None # None = ',' or pass str if something
        col_y_idx0 = 0 # col num (ind 0) that has predicted class from df
        dfx_col_idx0_drop_ls = [col_y_idx0] # y column and any other idx if need be
        bin_cols_dict = {1:2,2:2,3:2,4:2,5:2,6:2,7:2,8:2,9:2,10:5,11:5,12:5} # colname, bins
        
    if dataset=='glass':
        file_path = './data/glass/glass.data'
        header = None # None or 0 (first row)
        delimiter = None # None = ',' or pass str if something
        col_y_idx0 = 10 # col num (ind 0) that has predicted class from df
        dfx_col_idx0_drop_ls = [col_y_idx0, 0] # y column and any other idx if need be
        bin_cols_dict = {1:10,2:10,3:10,4:10,5:5,6:10,7:5,8:5} # colname, bins
        
    if dataset=='nist': # NEED TO FIX UP! only using training data here, not test file
        file_path = './data/nist/optdigits.tra'
        header = None # None or 0 (first row)
        delimiter = None # None = ',' or pass str if something
        col_y_idx0 = 64 # col num (ind 0) that has predicted class from df
        dfx_col_idx0_drop_ls = [col_y_idx0] # y column and any other idx if need be
        # bin_cols_dict = {1:5,2:5,3:5,4:5,5:5,6:5,7:5,8:5} # colname, bins
        bin_cols_dict = {x:4 for x in range(64)} # colname, bins

    if dataset=='post-operative':
        file_path = './data/post-operative/post-operative.data'
        header = None # None or 0 (first row)
        delimiter = None # None = ',' or pass str if something
        col_y_idx0 = 8 # col num (ind 0) that has predicted class from df
        dfx_col_idx0_drop_ls = [col_y_idx0] # y column and any other idx if need be
        bin_cols_dict = {} # colname, bins

    if dataset=='flag': # predicting language
        file_path = './data/flag/flag.data'
        header = None # None or 0 (first row)
        delimiter = None # None = ',' or pass str if something
        col_y_idx0 = 5 # col num (ind 0) that has predicted class from df
        dfx_col_idx0_drop_ls = [col_y_idx0, 0] # y column and any other idx if need be
        bin_cols_dict = {} # colname, bins
        
    if dataset=='balance':
        file_path = './data/balance/balance-scale.data'
        header = None # None or 0 (first row)
        delimiter = None # None = ',' or pass str if something
        col_y_idx0 = 0 # col num (ind 0) that has predicted class from df
        dfx_col_idx0_drop_ls = [col_y_idx0] # y column and any other idx if need be
        bin_cols_dict = {} # colname, bins
        
    if dataset=='hayes-roth':
        file_path = './data/hayes-roth/hayes-roth.data'
        header = None # None or 0 (first row)
        delimiter = None # None = ',' or pass str if something
        col_y_idx0 = 5 # col num (ind 0) that has predicted class from df
        dfx_col_idx0_drop_ls = [col_y_idx0, 0] # y column and any other idx if need be
        bin_cols_dict = {} # colname, bins
        
    if dataset=='lymphography':
        file_path = './data/lymphography/lymphography.data'
        header = None # None or 0 (first row)
        delimiter = None # None = ',' or pass str if something
        col_y_idx0 = 0 # col num (ind 0) that has predicted class from df
        dfx_col_idx0_drop_ls = [col_y_idx0] # y column and any other idx if need be
        bin_cols_dict = {} # colname, bins

    if dataset=='dermatology':
        file_path = './data/dermatology/dermatology.data'
        header = None # None or 0 (first row)
        delimiter = None # None = ',' or pass str if something
        col_y_idx0 = 34 # col num (ind 0) that has predicted class from df
        dfx_col_idx0_drop_ls = [col_y_idx0] # y column and any other idx if need be
        bin_cols_dict = {} # colname, bins

    if dataset=='lenses':
        file_path = './data/lenses/lenses.csv'
        header = None # None or 0 (first row)
        delimiter = None # None = ',' or pass str if something
        col_y_idx0 = 5 # col num (ind 0) that has predicted class from df
        dfx_col_idx0_drop_ls = [col_y_idx0, 0] # y column and any other idx if need be
        bin_cols_dict = {} # colname, bins
    
    # NOT USING ATM - too large to load
    if dataset=='cover':
        file_path = './data/cover/covtype.data'
        header = None # None or 0 (first row)
        delimiter = None # None = ',' or pass str if something
        col_y_idx0 = 54 # col num (ind 0) that has predicted class from df
        dfx_col_idx0_drop_ls = [col_y_idx0] # y column and any other idx if need be
        bin_cols_dict = {1:5,2:5,3:5,4:5,5:5,6:5,7:5,8:5, 9:5, 10:5} # colname, bins
    
    # load prepared data
    returned_data = data_prepare(file_path, header, col_y_idx0, dfx_col_idx0_drop_ls, 
                                 bin_cols_dict, delimiter, print_out, 
                                 test_size, k, n, seed_val)
    
    (df, dfx_train, dfx_test, dfy_train, dfy_test) = returned_data
    if print_out == 1:
        print(f"loaded: {dataset}")
    return df, dfx_train, dfx_test, dfy_train, dfy_test
    
# df, dfx_train, dfx_test, dfy_train, dfy_test = load_data('dermatology', print_out=0)
# df, dfx_train_ls, dfx_test_ls, dfy_train_ls, dfy_test_ls = load_data('dermatology', print_out=1, test_size=0.3, k=0)

""""
# IMPROVEMENT NOTES
- values 1 and 2 in a column turn into seperate columns instead of becoming 0 and 1? factorise first? test it
- how to better treat missing '?' values in the data sets?
- double check how the drop and binning works based on how you select y - when at beginning or at end
-- e.g. for flags y is in the middle - how does that affect the indexing?
- impute missing ? values when factorising? how would that work - bridges is like this

"""

'"\n# IMPROVEMENT NOTES\n- values 1 and 2 in a column turn into seperate columns instead of becoming 0 and 1? factorise first? test it\n- how to better treat missing \'?\' values in the data sets?\n- double check how the drop and binning works based on how you select y - when at beginning or at end\n-- e.g. for flags y is in the middle - how does that affect the indexing?\n- impute missing ? values when factorising? how would that work - bridges is like this\n\n'

In [95]:
def data_info():
    
    datasets = ['zoo', 'student', 'phishing', 'heart', 'wine', 
                'glass', 'nist', 'post-operative', 
                'flag', 'balance', 'hayes-roth', 'lymphography', 'dermatology'
                ,'lenses'
                ]
    
    for dataset in datasets:
        df, dfx_train, dfx_test, dfy_train, dfy_test = data_load(dataset, print_out=0)
        
        print(f'dataset: {dataset: <15} | y_classes: {dfy_train.shape[1]: <5} | x_features: {dfx_train.shape[1]: <5} | n-train: {dfx_train.shape[0]: <5} | n-test: {dfx_test.shape[0]: <5}')

    return dfy_train.shape[1], dfx_train.shape[1], dfx_train.shape[0], dfx_test.shape[0]
# data_info()

In [96]:
##### MODEL #####

In [97]:
def model_optimize(model, cb, m_console, m_time, threads, caller='model_optimize'):
    
    global log_folder # get log folder from global var

    # Gurobi-specific
    model.Params.LogToConsole = m_console
    model.Params.Threads = threads
    model.Params.TimeLimit = m_time # seconds
    model.Params.LogFile = log_folder + f"model_optimize-{caller}.log"
    lp_path = log_folder + f"model_optimize-{caller}.lp"
    # m.Params.IntFeasTol = 0.00000001  # make sure dec vars are integer (but extra work, disable if not needed)

    # lazyYN = 0 # to cut or not to cut

    if cb:
        log_model_optimize("Lazy Con: ON")
        model.Params.lazyConstraints = 1
        
        model.update()
        model.write(lp_path)
        model._vars = model.getVars() # has to be right before optimize (stores variables in object m)
        model.optimize(add_cuts)
        
        log_model_optimize(f"Optimised - caller: {caller}")

    else:
        log_model_optimize("Lazy Con: OFF")
        model.update()
        model.write(lp_path)
        model.optimize()
        
        log_model_optimize(f"Optimised - caller: {caller}")
        

In [98]:
def model_info_grb(model):

    # global m2
    
    info_all = {}
    info_all["objective"] = model.ObjVal
    info_all["bound"] = model.ObjBound
    # info_all["gap"] = model.MIPGap
    info_all["is_optimal"] = (model.status == GRB.OPTIMAL)
    info_all["num_nodes"] = model.NodeCount
    info_all["num_vars"] = model.NumIntVars + model.NumBinVars
    info_all["sol_count"] = model.SolCount

    if model.SolCount > 0:
        print("objective: %0.2f"%info_all["objective"])
        print("bound: %0.2f"%info_all["bound"])
        print("gap: %0.2f"%info_all["gap"])
        print("sol_count: %0.0f"%info_all["sol_count"])
        
    return info_all

In [99]:
def model_build(dfx, dfy, alpha=2, beta=5, beta_dict={}):
    
    ############################# INITIALISE MODEL #############################
    global m
    m = gp.Model('BLR')

    ############################# VARIABLES #############################
    ins_count = len(dfx) # number of instances in dataset
    # print(ins_count)

    ############################## OBJECTIVE CONTENTS #############################
    objective_ls = []

    ############################# BIAS #############################
    # testing
    # global bias_dict
    # global bias_bit_dict
    # global bias_bit_equ_dict
    # global bias_bit_class_equ_dict
    # global wpos_dict
    # global wneg_dict
    # global wnet_dict
    
    bias_dict = {}
    bias_bit_dict = {}
    bias_bit_equ_dict = {}
    bias_bit_class_equ_dict = {}

    # for each class
    for class_name2 in dfy: # one hot encoded so column names unique class names
        
        ### BIAS INTEGER ###
        
        # dec var
        bias = m.addVar(vtype=GRB.INTEGER, name=f"bias_c-{class_name2}")
        bias_dict[(class_name2)] = bias

        ### BIAS BITS ###
        
        # bounds
        
        feature_ls = [x for x in dfx]
        feature_count = len(feature_ls)
        bias_ub =  feature_count # has be multiples of 4 less 1 for Q to be whole
        bias_lb = -(feature_count+1)
        # print(f"bias_ub: {bias_ub}, bias_lb: {bias_lb}")
        bias_bit_class_equ = 0 # don't add bias_lb here! as added below!
        
        # Q formula
        Q_val = int(math.ceil(math.log2(bias_ub-bias_lb + 1)))
        # print(Q_val)

        
        for bit in range ( 1, Q_val + 1 ): # add to correct indexing for formula
            # print(bit)
            
            # DEC VAR
                
            # build formula
            bias_bit = m.addVar(vtype=GRB.BINARY, name=f"biasbit_c-{class_name2}_b-{bit}")
            bias_bit_equ = int( 2**(bit-1) ) * bias_bit
            
            # add to dicts for tracking
            bias_bit_dict[(class_name2, bit)] = bias_bit
            bias_bit_equ_dict[(class_name2, bit)] = bias_bit_equ
            
            # add to sumation
            bias_bit_class_equ = bias_bit_class_equ + bias_bit_equ

        # CONSTRAINT
        bias_bit_con1 = m.addConstr( bias_lb + bias_bit_class_equ <= bias_ub , name=f"biasbit1_con_c-{class_name2}")
        bias_bit_con2 = m.addConstr( bias_lb + bias_bit_class_equ == bias , name=f"biasbit2_con_c-{class_name2}")
        
        # add to dict for tracking
        bias_bit_class_equ_dict[(class_name2)] = bias_bit_class_equ

    ############################# PREDICTION FOR CLASS #############################
    y_ci_dict = {}

    # for each class
    for class_name3 in dfy: # one hot encoded so column names unique class names
        # for each instance
        for instance_num33 in range(ins_count):
            # dec var
            y_ci = m.addVar(vtype=GRB.INTEGER, name=f"y_ci_c-{class_name3}_i-{instance_num33}")
            y_ci_dict[(class_name3, instance_num33)] = y_ci
            
    ############################# MIS/CLASSIFICATION MARGIN #############################-
    epos_dict = {}
    eneg_dict = {}

    # for each instance
    for instance_num4 in range(ins_count):
        # dec var
        # epos = m.addVar(vtype=GRB.CONTINUOUS, name=f"epos_i-{instance_num4}") # was continous in original code
        epos = m.addVar(vtype=GRB.INTEGER, name=f"epos_i-{instance_num4}")
        epos_dict[(instance_num4)] = epos
        
        # eneg = m.addVar(vtype=GRB.CONTINUOUS, name=f"eneg_i-{instance_num4}") # was continous in original code
        eneg = m.addVar(vtype=GRB.INTEGER, name=f"eneg_i-{instance_num4}")
        eneg_dict[(instance_num4)] = eneg
        
        # con 7
        m.addConstr(epos >= 0, name=f"epos_con7_i-{instance_num4}")
        m.addConstr(eneg >= 0, name=f"eneg_con7_i-{instance_num4}")
        
        # add to objective list
        objective_ls.append( -1 * alpha * epos )
        
        # custom hyper parameter value
        if instance_num4 in beta_dict: # if have a custom value for this instance
            objective_ls.append( beta_dict[(instance_num4)] * eneg )
        else: # otherwise use standard
            objective_ls.append( beta * eneg )
        
    ############################# WEIGHTS (pos and neg and net) #############################
    wpos_dict = {}
    wneg_dict = {}
    wnet_dict = {}

    # wsub_dict = {}
    
    # for each feature
    for feature_name5 in dfx: # cols of dfx are the feature names (0+)
        # for each class
        for class_name55 in dfy: # one hot encoded so column names unique class names
            # dec var
            wpos = m.addVar(vtype=GRB.BINARY, name=f"wpos_f-{feature_name5}_c-{class_name55}")
            wpos_dict[(feature_name5, class_name55)] = wpos
            
            wneg = m.addVar(vtype=GRB.BINARY, name=f"wneg_f-{feature_name5}_c-{class_name55}")
            wneg_dict[(feature_name5, class_name55)] = wneg
            
            # wnet recorded for con3 later on
            wnet = wpos - wneg
            wnet_dict[(feature_name5, class_name55)] = wnet
                    
            # con 2
            m.addConstr(wpos + wneg <= 1, name=f"wcon2_f-{feature_name5}_c-{class_name55}")
            
            # add to objective list
            objective_ls.append(wpos)
            objective_ls.append(wneg)

    ############################# CONSTRAINT 3 + 4 #############################
    con3_dict = {}
    con4_dict = {}
    
    for instance_num6 in range(ins_count):
        for class_name66 in dfy: # one hot encoded so column names unique class names
            
            # CON 3
            con3_equ = 0
            
            # iterature features and sum
            for feature_name66 in dfx: # cols of dfx are the feature names (0+)
                if dfx[feature_name66][instance_num6] == 1: # only add weights where x_fi = 1 (instance and feature)
                    # con3_equ = con3_equ + wpos_dict[(feature_name66,class_name66)] - wnet_dict[(feature_name66,class_name66)]
                    con3_equ = con3_equ + wnet_dict[(feature_name66,class_name66)]
                
            # add bias for each class
            con3_equ = con3_equ + bias_dict[(class_name66)]
            
            # add contraint (equation = ypred)
            con3 = m.addConstr(con3_equ == y_ci_dict[(class_name66, instance_num6)] , name=f"con3_c-{class_name66}_i-{instance_num6}")
            con3_dict[(class_name66, instance_num6)] = con3
            
            
            # CON 4
            
            # if class does not equal the output class
            dfy_cold_enc = dfy.idxmax(axis=1) # undo 1 hot encoding
            y_label_class_name = dfy_cold_enc.loc[instance_num6]
            
            if class_name66 != y_label_class_name: # for all other (incorrect) classes
                
                con4 = m.addConstr( y_ci_dict[(y_label_class_name, instance_num6)] >= y_ci_dict[(class_name66, instance_num6)] + epos_dict[(instance_num6)] - eneg_dict[(instance_num6)] , name=f"con4_c-{class_name66}_i-{instance_num6}")
                con4_dict[(class_name66, instance_num6)] = con4
            

    ############################# OBJECTIVE #############################

    # test con (all weights cannot be 0!)
    # pos_con99 = m.addConstr( gp.quicksum( [v for v in wpos_dict.values()] ) >=1, name="pos_con99" )
    # neg_con99 = m.addConstr( gp.quicksum( [v for v in wneg_dict.values()] ) >=1, name="neg_con99" )
    
    objective_full = sum([x for x in objective_ls])
    m.setObjective(objective_full, GRB.MINIMIZE)
    
    return m, epos_dict, eneg_dict, wpos_dict, wneg_dict, wnet_dict, bias_dict, bias_bit_dict


In [100]:
# TEST model_build()
testing = 0
if testing:

    # df, dfx_train, dfx_test, dfy_train, dfy_test = data_load('student')
    # df, dfx_train, dfx_test, dfy_train, dfy_test = data_load('nist')
    # df, dfx_train, dfx_test, dfy_train, dfy_test = data_load('lenses')
    df, dfx_train, dfx_test, dfy_train, dfy_test = data_load('wine', print_out=1)
    # df, dfx_train, dfx_test, dfy_train, dfy_test = data_load('cover')

    alpha = 2
    beta = 5
    beta_dict = {1: 11, 2:22}
    ret_test_model_build = model_build(dfx_train, dfy_train, alpha, beta, beta_dict)
    (m, epos_dict, eneg_dict, wpos_dict, wneg_dict, wnet_dict, bias_dict, bias_bit_dict) = ret_test_model_build 
    cb = 0
    m_console = 0
    m_time = 5
    threads = 10
    model_optimize(m, cb, m_console, m_time, threads, caller='test_model_build')

    # Training + Test Accuracy
    training_accuracy, training_pred_outcome = model_accuracy(bias_dict, wpos_dict, wneg_dict, dfx_train, dfy_train)
    test_accuracy, test_pred_outcome = model_accuracy(bias_dict, wpos_dict, wneg_dict, dfx_test, dfy_test)
    print(f'Training Accuracy: {training_accuracy} | Test Accuracy: {test_accuracy}')

In [101]:
def add_cuts(m, where):
    if where == GRB.Callback.MIPSOL:
        
        print("cb")

        # build dicts using callback solution
        
        wpos_dict_val = {} # val as don't have to use .X
        wneg_dict_val = {} # val as don't have to use .X
        bias_dict_val = {} # val as don't have to use .X
        bias_bit_dict_val = {} # val as don't have to use .X
        
        wpos_dict_var = {} # vars w/o values
        wneg_dict_var = {} # vars w/o values
        bias_dict_var = {} # vars w/o values
        bias_bit_dict_var = {} # vars w/o values
        
        for fc1, var_name1 in wpos_dict.items(): # for (feature, class) in dict
            wpos_dict_val[fc1] = m.cbGetSolution(var_name1) # getting value, not dec var
            
            for var_cb in m._vars: # get vars from solution (the ones in dict you pass are not updated yet)
                if str(var_cb) == str(var_name1):
                    wpos_dict_var[fc1] = var_cb
                    # print(f"wpos_var: {var_cb}, type{type(var_cb)}")

        for fc2, var_name2 in wneg_dict.items():
            wneg_dict_val[fc2] = m.cbGetSolution(var_name2)

            for var_cb in m._vars:
                if str(var_cb) == str(var_name2):
                    wneg_dict_var[fc2] = var_cb
        
        for fc3, var_name3 in bias_dict.items():
            bias_dict_val[fc3] = m.cbGetSolution(var_name3)

            for var_cb in m._vars:
                if str(var_cb) == str(var_name3):
                    bias_dict_var[fc3] = var_cb
        
        for fc4, var_name4 in bias_bit_dict.items():
            bias_bit_dict_val[fc4] = m.cbGetSolution(var_name4)

            for var_cb in m._vars:
                if str(var_cb) == str(var_name4):
                    bias_bit_dict_var[fc4] = var_cb
        
        wcomb_dict_cb = model_wcomb(dfx_train, dfy_train, wpos_dict_val, wneg_dict_val, cb=True)
        
        # check if all weights null in solution (may be in initial solution) = True if null
        wcomb_null = all(value == 0 for value in wcomb_dict_cb.values())
        
        # if weights not null continue
        if wcomb_null:
            print("solution found : null weights")
            pass
        else:
            print("solution found : not-null weights")
            # print('wcomb_dict_cb:\n',wcomb_dict_cb)
            # print(wcomb_dict_cb)
            # check for adv example (status = 1 if found, else 0)
            ibudget = 3
            gbudget = 2
            msub_console = 0
            msub_time = 10
            itr_num = 999
            count_miss = 0
            score_threshold = 1
            adv_idx_ls = [1,2,3]
            wbudget = 3
            
            # pass val dict! 
            ret_adv_example_gen = adv_example_gen(dfx_train, dfy_train, wpos_dict, wneg_dict, bias_dict, bias_bit_dict, 
                                                  ibudget, adv_idx_ls, wbudget, msub_console, msub_time,
                                                  threads, itr_num, count_miss, score_threshold, cb=True)
            (adv_status, adv_count) = ret_adv_example_gen
            
            if adv_status == 1: # adv example found w/existing weights
                log_adv_train(f"Adv example found during itr_num: {itr_num} ")
                
                # generate cut
                # cut_equ_lhs, n_val_rhs = adv_cut(wpos_dict_val, wneg_dict_val, bias_bit_dict_val, cb=True)
                cut_equ_lhs, n_val_rhs = adv_cut_cb(wpos_dict_var, wneg_dict_var, bias_bit_dict_var,
                                                    wpos_dict_val, wneg_dict_val, bias_bit_dict_val)
                
                m.cbLazy( cut_equ_lhs <= n_val_rhs )
                log_adv_train(f" Lazy Con Added ")
                print(f" Lazy Con Added ")
                # print(cut_equ_lhs)
                # print(n_val_rhs)
                
                
            #     # add cut as constraint (to master model m)
            #     cut = m.addConstr( cut_equ_lhs <= nval_rhs , name=f"cut-{cut_num}" )
            #     cut_num = cut_num + 1
            #     cut_dict[cut_num] = cut
            #     log_adv_train(f"added cut : {cut_num}")
        
        print(adv_status, adv_count)

In [102]:
# TEST Bias Bits
testing = 0

# MAKE DICTS GLOBAL IN model_build() to test

if testing:
    
    # bias_bit_dict
    print("bias_dict")
    bias_val = 0
    for (k, v) in bias_dict.items():
        print(k,v.X)
    
    # bias bit class
    print("bias_bit_class_equ_dict")
    for k,v in bias_bit_class_equ_dict.items():
        print(k, v.getValue())
    
    # get bias val from bits
    print("bias_bit_dict")
    bias_val = 0
    for (k, v) in bias_bit_dict.items():
        print(k,v.X)
            

In [103]:
def model_wcomb(dfx, dfy, wpos_dict, wneg_dict, cb=False):
    
    # make weights dict (combining pos + neg weights)
    
    wcomb_dict = {}

    # for each feature
    for i1, feature_name1 in enumerate(dfx):
        
        # for each class
        for i2, class_name2 in enumerate(dfy): # one hot encoded so column names unique class names
            
            # when from callback already have value (no .X)
            if cb == False:
                
                if wpos_dict[(feature_name1, class_name2)].X == 1:
                    # print(feature_name3, class_name5)
                    wcomb_dict[feature_name1, class_name2] = 1
                    pass

                elif wneg_dict[(feature_name1, class_name2)].X == 1:
                    # print(feature_name3, class_name5)
                    wcomb_dict[feature_name1, class_name2] = -1
                    pass
                
                else: # both weights are zero
                    # print(feature_name3, class_name5)
                    wcomb_dict[feature_name1, class_name2] = 0
                    pass
                
            else:
                
                if wpos_dict[(feature_name1, class_name2)] == 1:
                    # print(feature_name3, class_name5)
                    wcomb_dict[feature_name1, class_name2] = 1
                    pass

                elif wneg_dict[(feature_name1, class_name2)] == 1:
                    # print(feature_name3, class_name5)
                    wcomb_dict[feature_name1, class_name2] = -1
                    pass
                
                else: # both weights are zero
                    # print(feature_name3, class_name5)
                    wcomb_dict[feature_name1, class_name2] = 0
                    pass        
                
            
    return wcomb_dict


In [104]:
def model_prediction(wcomb_dict, bias_dict, x_ins, class_ls, cb=False, ret='pred'):

        score_dict = {}

        for class_num in class_ls:
            class_score = 0
            
            # when from callback already have value (no .X)
            if cb == False:
                bias = bias_dict[(class_num)].X
            else:
                bias = bias_dict[(class_num)]
                
            class_score = class_score + bias
                
            for feature_num, feature_val in enumerate(x_ins):
                weight = wcomb_dict[(feature_num, class_num)]
                calc = feature_val * weight
                class_score = class_score + calc
                score_dict[class_num] = class_score
            
            # print(class_num, class_score)
            
        # print(max(score_dict, key=score_dict.get))

        if ret == 'score_dict':
            return score_dict
        elif ret == 'score_max':
            return max(score_dict.values()) # return value (score) of max score
        elif ret == 'pred':
            return max(score_dict, key=score_dict.get) # return key (class) of max score
        else:
            return 0

In [105]:
def model_accuracy(bias_dict, wpos_dict, wneg_dict, dfx, dfy):
    
    class_ls = [x for x in dfy]
    wcomb_dict = model_wcomb(dfx, dfy, wpos_dict, wneg_dict) # get weights combined
    ins_count = len(dfx)
    pred_outcome = []
    
    for i1, instance_num1 in enumerate( range(ins_count) ): # cycle through all training instances
        
        x_ins = list(dfx.loc[instance_num1]) # get instance
        x_ins_label = dfy.idxmax(axis=1)[instance_num1] # get label
        x_ins_pred = model_prediction(wcomb_dict, bias_dict, x_ins, class_ls, cb=False, ret='pred') # pred for x_ins
        # print(x_ins_label)
        # print(x_ins_pred)
        
        # compare
        if x_ins_pred == x_ins_label:
            pred_outcome.append(1.0)
        else:
            pred_outcome.append(0)

    decimal_accu = round( mean(pred_outcome) , 4 )
    percent_accu = round( mean(pred_outcome) * 100 , 2 )
    
    # print('Accuracy', mean(pred_outcome))
    return percent_accu, pred_outcome


In [106]:
def model_info_acc(bias_dict, wpos_dict, wneg_dict, dfx_train, dfy_train, dfx_test, dfy_test):
    
    train_acc, train_pred_outcome = model_accuracy(bias_dict, wpos_dict, wneg_dict, dfx_train, dfy_train)
    test_acc, test_pred_outcome = model_accuracy(bias_dict, wpos_dict, wneg_dict, dfx_test, dfy_test)
    
    print(f'Train Acc: {train_acc} | Test Acc: {test_acc} | Diff: {round(test_acc-train_acc,2)}')
    return train_acc, test_acc

In [107]:
##### ADV #####

In [108]:
## GET EXAMPLE ##

In [109]:
def adv_example_ins(q01_dict, q10_dict, x_ins):
    
    x_adv = []
    flips_ins = 0
    
    # for each x value
    for i, x in enumerate(x_ins):
        # print(x)
        # print( f"q01_dict[i].X: {q01_dict[i].X}")
        
        if q01_dict[i].X: # if 1
            # print(x)
            x_adv.append(1)
            flips_ins = flips_ins + 1
            
        elif q10_dict[i].X: # if 1
            x_adv.append(0)
            flips_ins = flips_ins + 1
        
        else:
            x_adv.append(x) # original value if no flip
            
    # print(x_ins)
    
    # print('flips:',no_flips)
    return x_adv, flips_ins
        
# x_ins_adv = get_adv_example(q01_dict, q10_dict, x_ins)
# print(x_ins)
# print(x_ins_adv)
    

In [110]:
## GET CUT ##

In [111]:
def adv_cut_cb(wpos_dict_var, wneg_dict_var, bias_bit_dict_var, 
               wpos_dict_val, wneg_dict_val, bias_bit_dict_val):
    """ build cut equation """
    # NOTE: val 1 -> w / val 0 -> 1-w <= n-1
    # NOTE: var and val in diff dicts now however key (feature, class) same of both = use that to build equ
    
    cut_equ_wpos = 0
    # get var + val (using var dict as base)
    for fc1 in wpos_dict_var: # fc1 is the (feature, class)
        wpos_var = wpos_dict_var[fc1] # get dec var
        wpos_val = int(wpos_dict_val[fc1]) # get val - callback does not need .X
    
        # build equation
        if wpos_val == 1:
            cut_equ_wpos = cut_equ_wpos + wpos_var
        elif wpos_val == 0:
            cut_equ_wpos = cut_equ_wpos + (1 - wpos_var)
        else:
            print("Issue w/Positive Cut Equ")
            print(f"wpos_dict_var: {wpos_dict_var}")
            print(f"wpos_dict_val: {wpos_dict_val}")
        
    cut_equ_wneg = 0
    # get var + val (using var dict as base)
    for fc2 in wneg_dict_var: # fc2 is the (feature, class)
        wneg_var = wneg_dict_var[fc2] # get dec var
        wneg_val = int(wneg_dict_val[fc2]) # get val - callback does not need .X

        # build equation
        if wneg_val == 1:
            cut_equ_wneg = cut_equ_wneg + wneg_var
        elif wneg_val == 0:
            cut_equ_wneg = cut_equ_wneg + (1 - wneg_var)
        else:
            print("Issue w/Negative Cut Equ")
            print(f"wneg_dict_var: {wneg_dict_var}")
            print(f"wneg_dict_val: {wneg_dict_val}")
    
    cut_equ_bias_bit = 0
    # get var + val (using var dict as base)
    for fc3 in bias_bit_dict_var: # fc2 is the (feature, class)
        bias_bit_var = bias_bit_dict_var[fc3] # get dec var
        bias_bit_val = int(bias_bit_dict_val[fc3]) # get val - callback does not need .X

        # build equation
        if bias_bit_val == 1:
            cut_equ_bias_bit = cut_equ_bias_bit + bias_bit_var
        elif bias_bit_val == 0:
            cut_equ_bias_bit = cut_equ_bias_bit + (1 - bias_bit_var)
        else:
            print("Issue w/Negative Cut Equ")
            print(f"wneg_dict_var: {bias_bit_dict_var}")
            print(f"wneg_dict_val: {bias_bit_dict_val}")

    # LHS
    # cut_equ_lhs = cut_equ_wpos + cut_equ_wneg + cut_equ_bias_bit
    cut_equ_lhs = cut_equ_wpos + cut_equ_wneg

    # RHS
    # n_val_rhs = len(wpos_dict_var) + len(wneg_dict_var) + len(bias_bit_dict_var) - 1
    n_val_rhs = len(wpos_dict_var) + len(wneg_dict_var) - 1

    # print(cut_equ_wpos)
    # print(cut_equ_wneg)
    # print(cut_equ_lhs)
    # print(n_val_rhs)
    
    return cut_equ_lhs, n_val_rhs

In [112]:
def adv_cut_v0(wpos_dict, wneg_dict, bias_bit_dict):
    """ build cut equation """
    # val 1 -> w / val 0 -> 1-w <= n-1
    
    int_sum = 0 # sum the integers (for checking)
    
    cut_equ_wpos = 0
    for k_pos, v_pos in wpos_dict.items():
        wpos_k = k_pos
        wpos_var = v_pos
        wpos_val = int(v_pos.X)
        
        if wpos_val == 1:
            cut_equ_wpos = cut_equ_wpos + wpos_var
        elif wpos_val == 0:
            cut_equ_wpos = cut_equ_wpos + (1 - wpos_var)
            int_sum = int_sum + 1
        else:
            print("Issue")
        # print(wpos_val)

    cut_equ_wneg = 0
    for k_neg, v_neg in wneg_dict.items():
        wneg_k = k_neg
        wneg_var = v_neg
        wneg_val = int(v_neg.X)
            
        if wneg_val == 1:
            cut_equ_wneg = cut_equ_wneg + wneg_var
        elif wneg_val == 0:
            cut_equ_wneg = cut_equ_wneg + (1 - wneg_var)
            int_sum = int_sum + 1
        else:
            print("Issue")
        # print(wneg_val)

    cut_equ_bias_bit = 0
    for k_bias, v_bias in bias_bit_dict.items():
        bias_k = k_bias
        bias_var = v_bias
        bias_val = int(v_bias.X)
        
        if bias_val == 1:
            cut_equ_bias_bit = cut_equ_bias_bit + bias_var
        elif bias_val == 0:
            cut_equ_bias_bit = cut_equ_bias_bit + (1 - bias_var)
            int_sum = int_sum + 1
        else:
            print("Issue")
            
        # print(bias_val)
        # print(bias_bit_dict)

    # LHS
    # cut_equ_lhs = cut_equ_wpos + cut_equ_wneg + cut_equ_bias_bit
    cut_equ_lhs = cut_equ_wpos + cut_equ_wneg

    # RHS
    # n_val_rhs = len(wpos_dict) + len(wneg_dict) + len(bias_bit_dict) - 1
    n_val_rhs = len(wpos_dict) + len(wneg_dict) - 1

    # print(cut_equ_wpos)
    # print(cut_equ_wneg)
    # print(cut_equ_lhs)
    # print(n_val)
    # print(int_sum)
    
    return cut_equ_lhs, n_val_rhs

In [113]:

def adv_cut_v1(wpos_dict, wneg_dict, bias_bit_dict, x_ins_label, x_adv_pred):
    """ build cut equation - only for label and adv class weights """
    # val 1 -> w / val 0 -> 1-w <= n-1
    
    int_sum = 0 # sum the integers (for checking)
    
    cut_equ_wpos = 0
    for k_pos, v_pos in wpos_dict.items():
        wpos_k = k_pos
        wpos_var = v_pos
        wpos_val = int(v_pos.X)
        wpos_feature = k_pos[0]
        wpos_class = k_pos[1]
        
        if wpos_class == x_ins_label or wpos_class == x_adv_pred:
            
            if wpos_val == 1:
                cut_equ_wpos = cut_equ_wpos + wpos_var
            elif wpos_val == 0:
                cut_equ_wpos = cut_equ_wpos + (1 - wpos_var)
                int_sum = int_sum + 1
            else:
                print("Issue")
            # print(wpos_val)

    cut_equ_wneg = 0
    for k_neg, v_neg in wneg_dict.items():
        wneg_k = k_neg
        wneg_var = v_neg
        wneg_val = int(v_neg.X)
        wneg_feature = k_neg[0]
        wneg_class = k_neg[1]

        if wneg_class == x_ins_label or wpos_class == x_adv_pred:

            if wneg_val == 1:
                cut_equ_wneg = cut_equ_wneg + wneg_var
            elif wneg_val == 0:
                cut_equ_wneg = cut_equ_wneg + (1 - wneg_var)
                int_sum = int_sum + 1
            else:
                print("Issue")
            # print(wneg_val)

    cut_equ_bias_bit = 0
    for k_bias, v_bias in bias_bit_dict.items():
        bias_k = k_bias
        bias_var = v_bias
        bias_val = int(v_bias.X)
        bias_feature = k_bias[0]
        bias_class = k_bias[1]
        
        if bias_class == x_ins_label or wpos_class == x_adv_pred:
            
            if bias_val == 1:
                cut_equ_bias_bit = cut_equ_bias_bit + bias_var
            elif bias_val == 0:
                cut_equ_bias_bit = cut_equ_bias_bit + (1 - bias_var)
                int_sum = int_sum + 1
            else:
                print("Issue")
                
            # print(bias_val)
            # print(bias_bit_dict)

    # LHS
    # cut_equ_lhs = cut_equ_wpos + cut_equ_wneg + cut_equ_bias_bit
    cut_equ_lhs = cut_equ_wpos + cut_equ_wneg # W/O BIAS

    # RHS
    # n_val_rhs = len(wpos_dict) + len(wneg_dict) + len(bias_bit_dict) - 1
    # n_val_rhs = len(wpos_dict) + len(wneg_dict) - 1
    n_val_rhs = cut_equ_lhs.size() - 1

    # print(cut_equ_wpos)
    # print(cut_equ_wneg)
    # print(cut_equ_lhs)
    # print(n_val)
    # print(int_sum)
    
    return cut_equ_lhs, n_val_rhs

In [114]:
def adv_cut_v2(dfx, dfy, x_ins, x_adv, x_ins_label, x_adv_pred, wpos_dict, 
               wneg_dict, bias_dict, bias_bit_dict, wbudget):
    """ build cut equation """
    
    # print(f"x_label:{x_ins_label} x_adv:{x_adv_pred}")
    global m_third
    m_third = gp.Model("Third-Problem")
    
    class_ls = [x for x in dfy]
    feature_ls = [x for x in dfx]
    
    # global q01pos_dict, q10pos_dict, q01neg_dict, q10neg_dict, q_vars_dict
    
    q01pos_dict = {} # q 0-1 flips
    q10pos_dict = {} # q 1-0 flips
    q01neg_dict = {} # q 0-1 flips
    q10neg_dict = {} # q 1-0 flips
    q_equ_other_dict = {} # q con equations for other classes (LHS)
    q_equ_label_dict = {}  # q con equations for predicted class (RHS)
    class_count = len(class_ls) # doesn't matter to include predicted class as gurobi will remove

    q_equ_adv = 0 # q con equation for adv example (LHS)
    q_equ_label = 0 # q con equation for label example (RHS)
    con_budget_equ = 0 # build budget equation summing over q's for each feature

    q_vars_dict = {}
    
    # WPOS
    
    ### PER FEATURE ### (q dec vars + constraints)
    for feature_name1 in feature_ls:
        # print(feature_name2)

        q_vars_name_ls = ['q01pos_adv', 'q01neg_adv','q10pos_adv','q10neg_adv', # adv
                          'q01pos_label','q01neg_label','q10pos_label', 'q10neg_label'] # label
        
        q_vars = m_third.addVars(q_vars_name_ls, vtype=GRB.BINARY, name=f"f-{feature_name1}")
        q_vars_dict[(feature_name1)] = q_vars
        # q constraints ------
        
        # adv
        
        # pos
        m_third.addConstr( wpos_dict[(feature_name1, x_adv_pred)].X + q_vars['q01pos_adv'] - q_vars['q10pos_adv'] <= 1, name=f"q1pos_adv_f-{feature_name1}")
        m_third.addConstr( wpos_dict[(feature_name1, x_adv_pred)].X + q_vars['q01pos_adv'] - q_vars['q10pos_adv'] >= 0, name=f"q0pos_adv_f-{feature_name1}")
        m_third.addConstr( q_vars['q01pos_adv'] + q_vars['q10pos_adv'] <= 1, name=f"qsumpos_adv_f-{feature_name1}")
        
        # neg
        m_third.addConstr( wneg_dict[(feature_name1, x_adv_pred)].X + q_vars['q01neg_adv'] - q_vars['q10neg_adv'] <= 1, name=f"q1neg_adv_f-{feature_name1}")
        m_third.addConstr( wneg_dict[(feature_name1, x_adv_pred)].X + q_vars['q01neg_adv'] - q_vars['q10neg_adv'] >= 0, name=f"q0neg_adv_f-{feature_name1}")
        m_third.addConstr( q_vars['q01neg_adv'] + q_vars['q10neg_adv'] <= 1, name=f"qsumneg_adv_f-{feature_name1}")
        
        # weight con (as in can't flip both from 0 to 1)
        m_third.addConstr( q_vars['q01pos_adv'] + q_vars['q01neg_adv'] <= 1, name=f"q01w_adv_f-{feature_name1}")
        
        # label
        
        # pos
        m_third.addConstr( wpos_dict[(feature_name1, x_ins_label)].X + q_vars['q01pos_label'] - q_vars['q10pos_label'] <= 1, name=f"q1pos_label_f-{feature_name1}")
        m_third.addConstr( wpos_dict[(feature_name1, x_ins_label)].X + q_vars['q01pos_label'] - q_vars['q10pos_label'] >= 0, name=f"q0pos_label_f-{feature_name1}")
        m_third.addConstr( q_vars['q01pos_label'] + q_vars['q10pos_label'] <= 1, name=f"qsumpos_label_f-{feature_name1}")

        # neg
        m_third.addConstr( wneg_dict[(feature_name1, x_ins_label)].X + q_vars['q01neg_label'] - q_vars['q10neg_adv'] <= 1, name=f"q1neg_label_f-{feature_name1}")
        m_third.addConstr( wneg_dict[(feature_name1, x_ins_label)].X + q_vars['q01neg_label'] - q_vars['q10neg_adv'] >= 0, name=f"q0neg_label_f-{feature_name1}")
        m_third.addConstr( q_vars['q01neg_label'] + q_vars['q10neg_adv'] <= 1, name=f"qsumneg_label_f-{feature_name1}")

        # weight con (as in can't flip both from 0 to 1)
        m_third.addConstr( q_vars['q01pos_label'] + q_vars['q01neg_label'] <= 1, name=f"q01w_label_f-{feature_name1}")
        
        # --------------------
        
        # build LHS and RHS equation
        q_equ_adv = q_equ_adv + ( (wpos_dict[(feature_name1, x_adv_pred)].X + q_vars['q01pos_adv'] - q_vars['q10pos_adv'] 
                                   - wneg_dict[(feature_name1, x_adv_pred)].X + q_vars['q01neg_adv'] - q_vars['q10neg_adv']) 
                                 * x_adv[feature_name1] ) # X ADV
        
        q_equ_label = q_equ_label + ( (wpos_dict[(feature_name1, x_ins_label)].X + q_vars['q01pos_label'] - q_vars['q10pos_label'] 
                                       - wneg_dict[(feature_name1, x_ins_label)].X + q_vars['q01neg_label'] - q_vars['q10neg_label']) 
                                     * x_ins[feature_name1] ) # X LABEL
        
        # wieght flip budget
        con_budget_equ = con_budget_equ + \
            q_vars['q01pos_adv'] + q_vars['q10pos_adv'] + \
            q_vars['q01neg_adv'] + q_vars['q10neg_adv'] + \
            q_vars['q01pos_label'] + q_vars['q10pos_label'] + \
            q_vars['q01neg_label'] + q_vars['q10neg_label']
                
    # Instance Level Dec Vars / Constraints (outside feature loop)
    score_diff = m_third.addVar(vtype=GRB.INTEGER, name=f"score_diff")
    m_third.addConstr( score_diff >= 0, name=f"score_diff_con_f-{feature_name1}") # larger than zero
    # m_third.addConstr( score_diff <= 5, name=f"score_diff_con2_f-{feature_name1}") # larger than zero
    
    # score con (add .X of bias as var is not in this model! + just need value)
    m_third.addConstr( q_equ_adv + bias_dict[(x_adv_pred)].X + score_diff >= q_equ_label + bias_dict[(x_ins_label)].X 
                      , name=f"score_con")
    
    m_third.update()
    
    # Overall Budget
    m_third.addConstr( con_budget_equ <= wbudget, name=f"qwbudget_con") # total number of flips
    # m_third.addConstr( con_budget_equ >= 1, name=f"qsumnonzero_con") # has to do something
    
    m_third.write(log_folder + 'adv_cutv2_mthird.lp')
    
    # objective
    m_third.setObjective(score_diff, GRB.MINIMIZE)
            
    # Gurobi-specific
    m_third.Params.LogToConsole = 0
    m_third.Params.Threads = 10
    m_third.Params.TimeLimit = 10
    m_third.Params.LogFile = log_folder + 'adv_cutv2_mthird.log'
    m_third.Params.PoolSearchMode = 2
    
    # m_third.Params.PoolSolutions = 100 # how many solutions to collect
    m_third.Params.PoolSolutions = glo_PoolSolutionsN # how many solutions to collect
    
    m_third.optimize()
    
    ########################### CYCLE SOLUTIONS ########################### 
    
    cut_equ_lhs_ls = []
    n_val_rhs_ls = []
    
    # no of solutions
    nSolutions = m_third.SolCount
    
    if nSolutions == 0:
        return 0,0
    
    log_adv_cut_v2(f"Number of solutions found (m_third): {nSolutions}")
    
    for e in range(nSolutions):
        m_third.setParam(GRB.Param.SolutionNumber, e)
        log_adv_cut_v2(f"Sol.No: {e}")
        
        wpos_dict_ls_adv = [int(v.X) for k, v in wpos_dict.items() if k[1] == x_adv_pred]
        wpos_dict_ls_label = [int(v.X) for k, v in wpos_dict.items() if k[1] == x_ins_label]

        wneg_dict_ls_adv = [int(v.X) for k, v in wneg_dict.items() if k[1] == x_adv_pred]
        wneg_dict_ls_label = [int(v.X) for k, v in wneg_dict.items() if k[1] == x_ins_label]

        wpos_dict_ls_adv_out = []
        for f, w in enumerate(wpos_dict_ls_adv):
            # print(f, w)
            if int(q_vars_dict[f]['q01pos_adv'].Xn) == 1:
                wpos_dict_ls_adv_out.append(1)
            elif int(q_vars_dict[f]['q10pos_adv'].Xn) == 1:
                wpos_dict_ls_adv_out.append(0)
            else:
                wpos_dict_ls_adv_out.append(w)

        wpos_dict_ls_label_out = []
        for f, w in enumerate(wpos_dict_ls_label):
            # print(f, w)
            if int(q_vars_dict[f]['q01pos_label'].Xn) == 1:
                wpos_dict_ls_label_out.append(1)
            elif int(q_vars_dict[f]['q10pos_label'].Xn) == 1:
                wpos_dict_ls_label_out.append(0)
            else:
                wpos_dict_ls_label_out.append(w)

        wneg_dict_ls_adv_out = []
        for f, w in enumerate(wneg_dict_ls_adv):
            # print(f, w)
            if int(q_vars_dict[f]['q01neg_adv'].Xn) == 1:
                wneg_dict_ls_adv_out.append(1)
            elif int(q_vars_dict[f]['q10neg_adv'].Xn) == 1:
                wneg_dict_ls_adv_out.append(0)
            else:
                wneg_dict_ls_adv_out.append(w)

        wneg_dict_ls_label_out = []
        for f, w in enumerate(wneg_dict_ls_label):
            # print(f, w)
            if int(q_vars_dict[f]['q01neg_label'].Xn) == 1:
                wneg_dict_ls_label_out.append(1)
            elif int(q_vars_dict[f]['q10neg_label'].Xn) == 1:
                wneg_dict_ls_label_out.append(0)
            else:
                wneg_dict_ls_label_out.append(w)
        
        
        # create weight comb dict for adv and label classes only
        wcomb_ls_adv_out = []
        for f in feature_ls:
            
            if wpos_dict_ls_adv_out[f] == 1:
                wcomb_ls_adv_out.append(1)
            
            elif wneg_dict_ls_adv_out[f] == 1:
                wcomb_ls_adv_out.append(-1)
            
            else:
                wcomb_ls_adv_out.append(0)
        
        wcomb_ls_label_out = []
        for f in feature_ls:
            
            if wpos_dict_ls_label_out[f] == 1:
                wcomb_ls_label_out.append(1)
            
            elif wneg_dict_ls_label_out[f] == 1:
                wcomb_ls_label_out.append(-1)
            
            else:
                wcomb_ls_label_out.append(0)
        
        log_adv_cut_v2(f"wpos_dict_ls_adv: {wpos_dict_ls_adv}")
        log_adv_cut_v2(f"wpos_dict_ls_adv_out: {wpos_dict_ls_adv_out}")
        log_adv_cut_v2(f"above 2 diff: {wpos_dict_ls_adv!=wpos_dict_ls_adv_out}")
        
        log_adv_cut_v2(f"wpos_dict_ls_label: {wpos_dict_ls_label}")
        log_adv_cut_v2(f"wpos_dict_ls_label_out: {wpos_dict_ls_label_out}")
        log_adv_cut_v2(f"above 2 diff: {wpos_dict_ls_label!=wpos_dict_ls_label_out}")
        
        log_adv_cut_v2(f"wneg_dict_ls_adv: {wneg_dict_ls_adv}")
        log_adv_cut_v2(f"wneg_dict_ls_adv_out: {wneg_dict_ls_adv_out}")
        log_adv_cut_v2(f"above 2 diff: {wneg_dict_ls_adv!=wneg_dict_ls_adv_out}")
        
        log_adv_cut_v2(f"wneg_dict_ls_label: {wneg_dict_ls_label}")
        log_adv_cut_v2(f"wneg_dict_ls_label_out: {wneg_dict_ls_label_out}")
        log_adv_cut_v2(f"above 2 diff: {wneg_dict_ls_label!=wneg_dict_ls_label_out}")
        
        log_adv_cut_v2(f"wcomb_ls_adv_out: {wcomb_ls_adv_out}")
        log_adv_cut_v2(f"wcomb_ls_label_out: {wcomb_ls_label_out}")
        log_adv_cut_v2(f"above 2 diff: {wcomb_ls_adv_out!=wcomb_ls_label_out}")
        
        # prediction
        class_score_adv = 0
        class_score_label = 0
        
        bias_adv = bias_dict[(x_adv_pred)].X
        bias_label = bias_dict[(x_ins_label)].X
            
        class_score_adv = class_score_adv + bias_adv
        class_score_label = class_score_label + bias_label
        
        # adv
        for feature_num, feature_val in enumerate(x_ins):
            calc = feature_val * wcomb_ls_adv_out[feature_num]
            class_score_adv = class_score_adv + calc
        # print(f"adv: {x_adv_pred}, score: {class_score_adv}")
        
        # label
        for feature_num, feature_val in enumerate(x_ins):
            calc = feature_val * wcomb_ls_label_out[feature_num]
            class_score_label = class_score_label + calc
        # print(f"label: {x_ins_label}, score: {class_score_label}")

        # build cut equ
        # global cut_equ_wpos, cut_equ_wneg
        
        cut_equ_wpos = 0
        for k, v in wpos_dict.items():
            wpos_k = k
            wpos_var = v
            wpos_feature = k[0]
            wpos_class = k[1]
            
            if wpos_class == x_adv_pred:
                
                if wpos_dict_ls_adv_out[wpos_feature] == 1: # no need for feature loop as will go through all anyway
                    cut_equ_wpos = cut_equ_wpos + wpos_var
                elif wpos_dict_ls_adv_out[wpos_feature] == 0:
                    cut_equ_wpos = cut_equ_wpos + (1 - wpos_var)
                else:
                    print("Issue")
                # print(wpos_val)
                
            if wpos_class == x_ins_label:
                
                if wpos_dict_ls_label_out[wpos_feature] == 1: # no need for feature loop as will go through all anyway
                    cut_equ_wpos = cut_equ_wpos + wpos_var
                elif wpos_dict_ls_label_out[wpos_feature] == 0:
                    cut_equ_wpos = cut_equ_wpos + (1 - wpos_var)
                else:
                    print("Issue")
                # print(wpos_val)

        cut_equ_wneg = 0
        for k, v in wneg_dict.items():
            wneg_k = k
            wneg_var = v
            wneg_feature = k[0]
            wneg_class = k[1]
            
            if wneg_class == x_adv_pred:
                
                if wneg_dict_ls_adv_out[wneg_feature] == 1: # no need for feature loop as will go through all anyway
                    cut_equ_wneg = cut_equ_wneg + wneg_var
                elif wneg_dict_ls_adv_out[wneg_feature] == 0:
                    cut_equ_wneg = cut_equ_wneg + (1 - wneg_var)
                else:
                    print("Issue")
                # print(wpos_val)
                
            if wneg_class == x_ins_label:
                
                if wneg_dict_ls_label_out[wneg_feature] == 1: # no need for feature loop as will go through all anyway
                    cut_equ_wneg = cut_equ_wneg + wneg_var
                elif wneg_dict_ls_label_out[wneg_feature] == 0:
                    cut_equ_wneg = cut_equ_wneg + (1 - wneg_var)
                else:
                    print("Issue")

        cut_equ_bias_bit = 0
        for k, v in bias_bit_dict.items():
            bias_k = k
            bias_var = v
            bias_val = int(v.X)
            bias_feature = k[0]
            bias_class = k[1]
            
            if bias_class == x_adv_pred:
                
                if bias_val == 1:
                    cut_equ_bias_bit = cut_equ_bias_bit + bias_var
                elif bias_val == 0:
                    cut_equ_bias_bit = cut_equ_bias_bit + (1 - bias_var)
                else:
                    print("Issue")
                # print(wpos_val)
                
            if bias_class == x_ins_label:
                
                if bias_val == 1:
                    cut_equ_bias_bit = cut_equ_bias_bit + bias_var
                elif bias_val == 0:
                    cut_equ_bias_bit = cut_equ_bias_bit + (1 - bias_var)
                else:
                    print("Issue")

        # ret equations to cut
        
        # cut_equ_lhs = cut_equ_wpos + cut_equ_wneg + cut_equ_bias_bit # W/BIAS CUT
        cut_equ_lhs = cut_equ_wpos + cut_equ_wneg # W/O BIAS CUT
        
        n_val_rhs = cut_equ_lhs.size() - 1 # isn't is just this? - size of lhs less 1?
        
        # append to output dict
        cut_equ_lhs_ls.append(cut_equ_lhs)
        n_val_rhs_ls.append(n_val_rhs)
        
    # print(f"adv", len(cut_equ_lhs_ls), len(n_val_rhs_ls))
    return cut_equ_lhs_ls, n_val_rhs_ls
    

In [115]:
## GEN EXAMPLE ##

In [116]:
def adv_example_gen(dfx, dfy, wpos_dict, wneg_dict, bias_dict, bias_bit_dict, ibudget, adv_idx_ls, wbudget,
                    msub_console, msub_time, threads, itr_num, count_miss, score_threshold, cb=False):
    # Params
    class_ls = [x for x in dfy]
    feature_ls = [x for x in dfx]
    wcomb_dict = model_wcomb(dfx, dfy, wpos_dict, wneg_dict, cb)
    adv_count = 0           # count num of adv examples found / used for gbudget below
    missclass_count = 0      # count num of misclassified samples
    
    # print(ins_count)
    # print(wcomb_dict)
    # print(len(wcomb_dict))
    
    # index of adv examples from dfx the sub sees
    # adv_idx_ls = [2,3]
    # ins_count = len(adv_idx_ls)    # number of instances in adv_idx passed
    
    for instance_num1 in adv_idx_ls : # cycle through all training instances 
        
        # get x instance details
        x_ins = list(dfx.loc[instance_num1])
        x_ins_label = dfy.idxmax(axis=1)[instance_num1]
        
        # get instance prediction based on current weights/bias
        x_ins_pred = model_prediction(wcomb_dict, bias_dict, x_ins, class_ls, cb, ret='pred')
        x_ins_pred_score = model_prediction(wcomb_dict, bias_dict, x_ins, class_ls, cb, ret='score_max')
        x_ins_pred_score_dict = model_prediction(wcomb_dict, bias_dict, x_ins, class_ls, cb, ret='score_dict')
        
        # log
        log_adv_example_gen(f'################## itr_num: {itr_num} ###################') # itr_num passed from adv_train() to keep track
        log_adv_example_gen(f'------------------ Instance Details ------------------') # itr_num passed from adv_train() to keep track
        log_adv_example_gen(f'instance_num1: {instance_num1}')
        log_adv_example_gen(f'label: {x_ins_label}')
        log_adv_example_gen(f'pred: {x_ins_pred}')
        log_adv_example_gen(f'correct: {x_ins_pred==x_ins_label}')
        log_adv_example_gen(f'max class score: {x_ins_pred_score}')
        log_adv_example_gen(f'all class scores: {x_ins_pred_score_dict}')
        
        if cb:
            print(f'------------------ Instance Details ------------------') # itr_num passed from adv_train() to keep track
            print(f'instance_num1: {instance_num1}')
            print(f'label: {x_ins_label}')
            print(f'pred: {x_ins_pred}')
            print(f'correct: {x_ins_pred==x_ins_label}')
            print(f'max class score: {x_ins_pred_score}')
            print(f'all class scores: {x_ins_pred_score_dict}')
    
        if int(x_ins_pred) != int(x_ins_label): # if misclassfied sample
             
            missclass_count = missclass_count + 1
            continue
         
        if int(x_ins_pred) == int(x_ins_label): # if correctly classified sample
                        
            # create model
            global m_sub
            m_sub = gp.Model("Sub-Problem")

            q01_dict = {} # q 0-1 flips
            q10_dict = {} # q 1-0 flips
            z_ind_dict = {} # dict for indicator vars
            q_equ_other_dict = {} # q con equations for other classes (LHS)
            q_equ_label_dict = {}  # q con equations for predicted class (RHS)
            class_count = len(class_ls) # doesn't matter to include predicted class as gurobi will remove
            
            ### PER FEATURE ### (q dec vars + constraints)
            for feature_name2 in feature_ls:
                # print(feature_name2)
                
                # Dec Var
                q01 = m_sub.addVar(vtype=GRB.BINARY, name=f"q01_f-{feature_name2}")
                q01_dict[(feature_name2)] = q01

                q10 = m_sub.addVar(vtype=GRB.BINARY, name=f"q10_f-{feature_name2}")
                q10_dict[(feature_name2)] = q10
                
                # print(feature_name2, x_ins[feature_name2])
                
                # q flip constraints
                m_sub.addConstr( x_ins[feature_name2] + q01 - q10 <= 1, name=f"q1_con_f-{feature_name2}")
                m_sub.addConstr( x_ins[feature_name2] + q01 - q10 >= 0, name=f"q0_con_f-{feature_name2}")
                m_sub.addConstr( q01 + q10 <= 1, name=f"qsum_con_f-{feature_name2}")
                m_sub.update()
                
            ### PER CLASS ### (build LHS and RHS equations)
            for i3, class_name3 in enumerate(class_ls):
                
                # print(f"i3: {i3}, class_name3: {class_name3}")
                
                q_equ_other = 0 # q con equation for other classes (LHS)
                q_equ_label = 0 # q con equation for label class (RHS)
                
                # label class (adv)
                if int(class_name3) == int(x_ins_label):
                    continue # move onto next class as don't need to build equation for label class
                
                # for other classes
                else:
                    for i4, feature_name4 in enumerate(feature_ls): 
                        
                        # x*q*wgt for label class
                        wgt_f_c_label = wcomb_dict[feature_name4, x_ins_label] # use x ins label
                        q_equ_label = q_equ_label + (x_ins[feature_name4] + q01_dict[feature_name4] - q10_dict[feature_name4]) * wgt_f_c_label
                        
                        # x*q*wgt for other classes                  
                        wgt_f_c_other = wcomb_dict[feature_name4, class_name3] # use class iterating
                        q_equ_other = q_equ_other + (x_ins[feature_name4] + q01_dict[feature_name4] - q10_dict[feature_name4]) * wgt_f_c_other

                    # add bias for label class (if outside cb use .X)
                    if cb:
                        bias_label = bias_dict[(x_ins_label)] # use x_ins_label & bias val dict for cb
                    else:
                        bias_label = bias_dict[(x_ins_label)].X # use x_ins_label & bias val dict for cb
                    q_equ_label = q_equ_label + bias_label
                    q_equ_label_dict[x_ins_label] = q_equ_label
                    
                    # add bias for other classes (if outside cb use .X)
                    if cb:
                        bias_other = bias_dict[(class_name3)] # use class iterating & bias val dict
                    else:
                        bias_other = bias_dict[(class_name3)].X # use class iterating & bias val dict
                    q_equ_other = q_equ_other + bias_other
                    q_equ_other_dict[class_name3] = q_equ_other
                    
                    # add score constraint on other classes w/indicator
                    z = m_sub.addVar( vtype="B", name=f"ind[{class_name3}]")
                    z_ind_dict[class_name3] = z
                    m_sub.addConstr( (z == 1) >> (q_equ_other + score_threshold >= q_equ_label) , name=f"score_con_c-{class_name3}")                        
                    
            # at least 1 z active
            m_sub.addConstr(gp.quicksum( [x[1] for x in z_ind_dict.items()] ) >= 1, name="select")
            
            # total q flip budget (per instance)
            con_budget_equ = 0 # build budget equation summing over q's for each feature
            for feature_name5 in feature_ls:
                con_budget_equ = con_budget_equ + q01_dict[feature_name5] + q10_dict[feature_name5]
            m_sub.addConstr( con_budget_equ <= ibudget, name=f"qbudget_con")

            # TEST CON (all q cannot be zero! cannot be 0!)
            m_sub.addConstr( con_budget_equ >= 1, name=f"qsumnonzero_con")
            
            # Set Objective (for testing)
            # m_sub.setObjective(con_budget_equ, GRB.MINIMIZE) # minimise flips
            # m_sub.setObjective(con_budget_equ, GRB.MAXIMIZE) # minimise flips
            # m_sub.setObjective(q_equ_other, GRB.MAXIMIZE)
            # m_sub.setObjective(q_equ_other + score_threshold - q_equ_label, GRB.MAXIMIZE)
            
            # m_sub.update()
            m_sub.write(log_folder + 'adv_example_gen_msub.lp')
            
            # Gurobi-specific
            m_sub.Params.LogToConsole = msub_console
            m_sub.Params.Threads = threads
            m_sub.Params.TimeLimit = msub_time
            m_sub.Params.LogFile = log_folder + 'adv_example_gen_msub.log'
            # m_sub.Params.PoolSearchMode = 2
            # m_sub.Params.PoolSolutions = 20
            m_sub.optimize()
            
            log_adv_example_gen("-----------Sub-Problem Details-----------")
            
            if m_sub.SolCount == 0:
                log_adv_example_gen(f"No solution found")
                log_adv_example_gen(f"x_ins: {x_ins}")
                log_adv_example_gen(f"adv_count (unchanged): {adv_count}")
                
                if cb:
                    print(f"No solution found")
                    print(f"x_ins: {x_ins}")
                    print(f"adv_count (unchanged): {adv_count}")
                continue
            
            else: # if solution found
                
                # nSolutions = m_sub.SolCount
                # print('Number of solutions found (m_sub): ' + str(nSolutions))
                log_adv_example_gen(f"A solution found")
                
                # get solution details + prediction
                x_adv, no_flips = adv_example_ins(q01_dict, q10_dict, x_ins) # get the x_adv (adv version of x_ins)
                x_adv_pred = model_prediction(wcomb_dict, bias_dict, x_adv, class_ls, cb, ret='pred') # use x_adv
                x_adv_pred_score_dict = model_prediction(wcomb_dict, bias_dict, x_adv, class_ls, cb, ret='score_dict')

                log_adv_example_gen(f"x_ins: {x_ins}")
                log_adv_example_gen(f"x_adv: {x_adv}")
                log_adv_example_gen(f"flips: {no_flips}")
                
                log_adv_example_gen(f"x_ins_pred: {x_ins_pred}")
                log_adv_example_gen(f"x_ins_pred score_dict: {x_ins_pred_score_dict}")
                
                log_adv_example_gen(f"x_adv_pred: {x_adv_pred}")
                log_adv_example_gen(f"x_adv_pred score_dict: {x_adv_pred_score_dict}")
                
                if cb: # print
                    print(f"A solution found")
                    print(f"x_ins: {x_ins}")
                    print(f"x_adv: {x_adv}")
                    print(f"flips: {no_flips}")
                    print(f"x_ins_pred: {x_ins_pred}")
                    print(f"x_ins_pred score_dict: {x_ins_pred_score_dict}")
                    print(f"x_adv_pred: {x_adv_pred}")
                    print(f"x_adv_pred score_dict: {x_adv_pred_score_dict}")
                
                ### SHOW WEIGHTS (for testing/checking)
                wpos_label_ls = []
                wneg_label_ls = []
                wpos_adv_ls = []
                wneg_adv_ls = []
                q01_ls = []
                q10_ls = []
                
                if not cb: # doesn't work in cb
                    
                    for f_c, var in wpos_dict.items():
                        if f_c[1] == x_ins_label:
                            wpos_label_ls.append( int(var.X) )
                        if f_c[1] == x_adv_pred:
                            wpos_adv_ls.append( int(var.X) )
                        
                    for f_c, var in wneg_dict.items():
                        if f_c[1] == x_ins_label:
                            wneg_label_ls.append( int(var.X) )
                        if f_c[1] == x_adv_pred:
                            wneg_adv_ls.append( int(var.X) )
                            
                    for f, var in q01_dict.items():
                            q01_ls.append( int(var.X) )
                            
                    for f, var in q10_dict.items():
                            q10_ls.append( int(var.X) ) 
                    
                    log_adv_example_gen(f"label wpos ls: {wpos_label_ls}")
                    log_adv_example_gen(f"label wneg ls: {wneg_label_ls}")
                    log_adv_example_gen(f"label bias: {bias_dict[(x_ins_label)].X}")
                    
                    log_adv_example_gen(f"adv wpos ls: {wpos_adv_ls}")
                    log_adv_example_gen(f"adv wneg ls: {wneg_adv_ls}")
                    log_adv_example_gen(f"adv bias: {bias_dict[(x_adv_pred)].X}")
                    
                    log_adv_example_gen(f"label+adv w+b: {wpos_label_ls}|{wneg_label_ls}|{bias_dict[(x_ins_label)].X}|{wpos_adv_ls}|{wneg_adv_ls}|{bias_dict[(x_adv_pred)].X}")
                    
                    log_adv_example_gen(f"q01: {q01_ls}")
                    log_adv_example_gen(f"q10: {q10_ls}")
                    
                # check if solution changes the prediction
                if int(x_ins_label) == int(x_adv_pred):
                    log_adv_example_gen(f"Solution does not change prediction")
                    log_adv_example_gen(f"adv_count (unchanged): {adv_count}")
                    
                    if cb: # print
                        print(f"Solution does not change prediction")
                        print(f"adv_count (unchanged): {adv_count}")
                    
                    continue # not an adv example and continue loop
                
                else:
                    adv_count = adv_count + 1 # HAS TO RUN EVEN IF CALL BACK
                    log_adv_example_gen(f"Solution does change prediction")
                    log_adv_example_gen(f"adv_count (incremented): {adv_count}")
                    
                    if cb: # print
                        print(f"Solution does change prediction")
                        print(f"adv_count (incremented): {adv_count}")
                    
                    # global record of count of examples (for testing)
                    glo_adv_ex_dict[instance_num1] = glo_adv_ex_dict.get(instance_num1, 0) + 1
                        
                    # get cuts
                    cut_equ_lhs_ls = []
                    nval_rhs_ls = []
                    
                    if glo_cut_v0 == True:
                        # cut v0 (add to list as returning cuts list now)
                        cut_equ_lhs, nval_rhs = adv_cut_v0(wpos_dict, wneg_dict, bias_bit_dict)
                        cut_equ_lhs_ls.append(cut_equ_lhs)
                        nval_rhs_ls.append(nval_rhs)
                    
                    if glo_cut_v1 == True:
                        # cut v0 (add to list as returning cuts list now)
                        cut_equ_lhs, nval_rhs = adv_cut_v1(wpos_dict, wneg_dict, bias_bit_dict, x_ins_label, x_adv_pred)
                        cut_equ_lhs_ls.append(cut_equ_lhs)
                        nval_rhs_ls.append(nval_rhs)
                    
                    if glo_cut_v2 == True:
                        cut_equ_lhs_ls_wip, nval_rhs_ls_wip = adv_cut_v2(dfx, dfy, x_ins, x_adv, x_ins_label, x_adv_pred,
                                                                wpos_dict, wneg_dict, bias_dict, bias_bit_dict, wbudget)
                        cut_equ_lhs_ls.extend(cut_equ_lhs_ls_wip)
                        nval_rhs_ls.extend(nval_rhs_ls_wip)    
                    
                    return 1, adv_count, cut_equ_lhs_ls, nval_rhs_ls # returns as soon as adv example found
    
    return 0, adv_count, [], [] # no adv example found (first return 0 and last adv_count)

In [117]:
# TEST adv_example_gen()
testing = 0
if testing:
    
    clear_log_adv_example_gen()
    clear_log_adv_train()
    clear_log_model_optimize()
    
    # dataset = 'balance'
    dataset = 'lenses'
    # dataset = 'zoo'
    count_miss = 0
    ibudget = 25
    wbudget = 2
    adv_idx_ls = [10]
    # gbudget = 2000 # global budget / should return with 1 more
    df, dfx_train, dfx_test, dfy_train, dfy_test = data_load(dataset, print_out=1)
    
    # build original model
    alpha = 2
    beta = 5
    
    # optimise original model to get
    cb = 0 
    m_console = 0
    m_time = 15
    threads = 10
    msub_console = 0
    msub_time = 10
    threads=6
    itr_num=-1
    score_threshold = 1
    
    global glo_cut_v0 # all weights
    global glo_cut_v1 # only adv and label class
    global glo_cut_v2 # combos of adv and label class
    global glo_adv_ex_dict

    glo_cut_v0 = False
    glo_cut_v1 = False
    glo_cut_v2 = False
    glo_adv_ex_dict = {}

    new_master_model = 1 # if want to rebuild or use existing global master model
    if new_master_model:
        # build master
        ret_test_adv_example = model_build(dfx_train, dfy_train, alpha, beta)
        (m, epos_dict, eneg_dict, wpos_dict, wneg_dict, wnet_dict, bias_dict, bias_bit_dict) = ret_test_adv_example 
        # optimise master
        model_optimize(m, cb, m_console, m_time, threads)

    # Training + Test Accuracy
    training_accuracy, training_pred_outcome = model_accuracy(bias_dict, wpos_dict, wneg_dict, dfx_train, dfy_train)
    test_accuracy, test_pred_outcome = model_accuracy(bias_dict, wpos_dict, wneg_dict, dfx_test, dfy_test)
    print(f'Training Accuracy: {training_accuracy} | Test Accuracy: {test_accuracy}')
    
    
    ret_adv_status = adv_example_gen(dfx_train, dfy_train, 
                                 wpos_dict, wneg_dict, bias_dict, bias_bit_dict,
                                 ibudget, adv_idx_ls, wbudget,
                                 msub_console, msub_time, threads, itr_num, count_miss, 
                                 score_threshold, cb=False)
    
    (adv_status, adv_count, cut_equ_lhs_ls, nval_rhs_ls) = ret_adv_status
    
    print(adv_count)
    print(cut_equ_lhs_ls)
    print(nval_rhs_ls)
    

In [118]:
##### TRAIN #####

In [119]:
def adv_train(dfx, dfy, alpha, beta, ibudget, adv_idx_ls, wbudget, m_time, msub_time, threads, max_itr, 
              m_console, msub_console, cb, c_time, count_miss, score_threshold):

    # build master model
    returned_model = model_build(dfx, dfy, alpha, beta)
    (m, epos_dict, eneg_dict, wpos_dict, wneg_dict, wnet_dict, bias_dict, bias_bit_dict) = returned_model
    
    # records
    cut_dict = {}
    cut_num = 0
    wpos_upd_dict = {} # pos wghts for each iteration
    wneg_upd_dict = {} # neg wghts for each iteration
    itr_num = 0 # count iterations
    
    # breaking criterion
    robust = False
    
    # for time out / time print
    start_time = time.time() # time now when loop started
    s=time.gmtime(start_time)
    start_time_f = time.strftime("%Y-%m-%d %H:%M:%S", s)
    
    end_time = time.time() + c_time # time now + time out secs = when loop should end
    e=time.gmtime(end_time)
    end_time_f = time.strftime("%Y-%m-%d %H:%M:%S", e)
    
    # initial optimization
    model_optimize(m, cb, m_console, m_time, threads, caller='adv_train')
    log_adv_train(f"initial optimization")
    log_adv_train(f"while loop timeout: {c_time}s")
    log_adv_train(f"while loop start_time: {start_time_f}")
    log_adv_train(f"while loop end_time: {end_time_f}")
    
    while robust == False:
        # time.sleep(0.25) # sleep for 250 milliseconds
        itr_num = itr_num + 1
        log_adv_train(f'################## itr_num: {itr_num} time: {int(time.time() - start_time)}s ###################')
        
        # precut copy of wght + bias dicts (for testing/checking)
        wpos_dict_precut = {}
        wneg_dict_precut = {}
        bias_dict_precut = {}

        for f_c, var in wpos_dict.items():
            wpos_dict_precut[f_c] = var.X
                
        for f_c, var in wneg_dict.items():
            wneg_dict_precut[f_c] = var.X
                
        for f_c, var in bias_dict.items():
            bias_dict_precut[f_c] = var.X

        # check for adv example (status = 1 if found, else 0)
        ret_adv_example_gen = adv_example_gen(dfx, dfy, 
                                              wpos_dict, wneg_dict, bias_dict, bias_bit_dict, 
                                              ibudget, adv_idx_ls, wbudget,
                                              msub_console, msub_time, threads, itr_num, 
                                              count_miss, score_threshold, cb=False)
        (adv_status, adv_count, cut_equ_lhs_ls, nval_rhs_ls) = ret_adv_example_gen
        
        if adv_status == 1: # adv example found w/existing weights
            log_adv_train(f"Adv example found during itr_num: {itr_num} ")
            
            # add cut as constraint (to master model m)
            # cut_equ_lhs, nval_rhs = adv_cut(wpos_dict, wneg_dict, bias_bit_dict)
            # cut = m.addConstr( cut_equ_lhs <= nval_rhs , name=f"cutv1-{cut_num}" )
            # cut_dict[cut_num] = cut
            # cut_num = cut_num + 1
            
            # log_adv_train(f"added cut : {cut_num}")
            
            # cuts
            for cut_idx in range ( len(cut_equ_lhs_ls) ):
                cut = m.addConstr( cut_equ_lhs_ls[cut_idx] <= nval_rhs_ls[cut_idx] , name=f"cut_idx-{cut_idx}" )
                cut_dict[cut_num] = cut
                cut_num = cut_num + 1
                pass
            
            log_adv_train(f"added cuts : {cut_num}")
                
            # optimise w/new constraint
            model_optimize(m, cb, m_console, m_time, threads, caller='adv_train')
            log_adv_train(f"re-optimized")
            log_adv_train(f"ObjVal: {m.ObjVal} | Bound: {m.ObjBound} | Gap: {m.MIPGap} | Optimal: {m.Status==GRB.OPTIMAL}")

            # create post-cut dict (for testing/checking)
            wpos_dict_postcut = {}
            wneg_dict_postcut = {}
            bias_dict_postcut = {}

            for f_c, var in wpos_dict.items():
                wpos_dict_postcut[f_c] = var.X
                
            for f_c, var in wneg_dict.items():
                wneg_dict_postcut[f_c] = var.X

            for f_c, var in bias_dict.items():
                bias_dict_postcut[f_c] = var.X
                
            # check + report diff in weights
            log_adv_train("--------------------- wgt details ---------------------")
            for f_c in wpos_dict:
                if wpos_dict_postcut[f_c] != wpos_dict_precut[f_c]:
                    log_adv_train(f"wpos diff (f,c): {f_c} | precut: {wpos_dict_precut[f_c]} | postcut: {wpos_dict_postcut[f_c]}")

            for f_c in wneg_dict:
                if wneg_dict_postcut[f_c] != wneg_dict_precut[f_c]:
                    log_adv_train(f"wneg diff (f,c): {f_c} | precut: {wneg_dict_precut[f_c]} | postcut: {wneg_dict_postcut[f_c]}")
            
            for f_c in bias_dict:
                if bias_dict_postcut[f_c] != bias_dict_precut[f_c]:
                    log_adv_train(f"bias diff (f,c): {f_c} | precut: {bias_dict_precut[f_c]} | postcut: {bias_dict_postcut[f_c]}")

            
            #################### check break criterion ####################
            # log_adv_train("--------- loop details ---------")
            
            if max_itr != -1: # max iteration specified (-1 would mean no max)
                if itr_num == max_itr:
                    log_adv_train(f"return from while loop at end of itr_num: {itr_num}")
                    return m, wpos_dict, wneg_dict, bias_dict, cut_dict
            
            if c_time != 0: # break after timeout if set (0 no timeout)
                if time.time() > end_time: # if time now goes past endtime break
                    log_adv_train(f"return from while loop after: {int(time.time() - start_time)}s at end of itr_num: {itr_num}")
                    return m, wpos_dict, wneg_dict, bias_dict, cut_dict
            
            # otherwise continue loop
            continue
            
        ###############################################################
        
        else: # no adv example found hence return
            log_adv_train(f"return from while loop at end of itr_num: {itr_num} after achieving ROBUSTNESS")
            print("---\nROBUST ARE WE---")
            robust = True
            return m, wpos_dict, wneg_dict, bias_dict, cut_dict # so can use in rest of code for testing/checking


In [120]:
# TEST adv_train()
testing = 0

if testing:

    # clear logs
    clear_log_adv_example_gen()
    clear_log_adv_train()
    clear_log_model_optimize()

    # Load Data
    # dataset = 'wine'
    # dataset = 'balance'
    dataset = 'zoo'
    df, dfx_train, dfx_test, dfy_train, dfy_test = data_load(dataset, print_out=1)

    # Adv Train with max_itr > 0 / -1 for no max / 0 for non-adv training
    max_itr = 5
    adv_idx_ls = [1,3,5]
    ibudget = 2
    wbudget = 5
    count_miss = 0
    cb = 0 # not ready yet
    score_threshold = 1

    global glo_cut_v0
    global glo_cut_v1
    global glo_cut_v2
    global glo_adv_ex_dict

    glo_cut_v0 = True
    glo_cut_v1 = False
    glo_cut_v2 = False
    glo_adv_ex_dict = {}

    m_time = 5
    msub_time = 5
    threads = 10
    c_time = 0 # max cutting time in secods (0 = no timeout)

    m_console = 0
    msub_console = 0

    alpha = 2
    beta = 5
    
    try:
        
        ret_adv_train = adv_train(dfx_train, dfy_train, alpha, beta, 
                                  ibudget, adv_idx_ls, wbudget,
                                  m_time, msub_time, threads, max_itr, 
                                  m_console, msub_console, cb, c_time, count_miss, 
                                  score_threshold)

        (m, wpos_dict, wneg_dict, bias_dict, cut_dict) = ret_adv_train
    
    except KeyboardInterrupt:
        print('Interupt')
    # push(title=f"d:{dataset} done on {pc_name}")
    train_acc, test_acc = model_info_acc(bias_dict, wpos_dict, wneg_dict, dfx_train, dfy_train, dfx_test, dfy_test)

In [121]:
##### MAIN #####

In [122]:
def main(record, dataset, n, max_itr, ibudget, adv_idx_ls, wbudget, alpha, beta, beta_dict, 
         score_threshold, m_time, m_console, c_time, test_size, seed_val):
    
    global df, dfx_train, dfx_test, dfy_train, dfy_test
    
    rec_dict = {}
    rec_dict['status'] = 'TBD' # put in first then update below
    
    start_time_main = datetime.now()
    start_timer = time.perf_counter()
    rec_dict['start_time_main'] = start_time_main.strftime('%Y-%m-%d_%H-%M-%S')
    
    rec_dict['version'] = 1.1
    rec_dict['notes'] = ''
    rec_dict['pc_name'] = pc_name
    
    if pc_name == 'Griffon':
        rec_dict['threads'] = 6
        
    if pc_name == 'Henry':
        rec_dict['threads'] = 8
    
    if pc_name == 'Elderax':
        rec_dict['threads'] = 10
    
    if pc_name == 'Queen':
        rec_dict['threads'] = 16

    # load data
    
    rec_dict['dataset'] = dataset
    # rec_dict['dataset'] = 'post-operative'
    # rec_dict['dataset'] = 'zoo'

    rec_dict['print_out_data'] = 1
    df, dfx_train, dfx_test, dfy_train, dfy_test = data_load(dataset=rec_dict['dataset'], 
                                                             print_out=rec_dict['print_out_data'],k=0,
                                                             test_size=test_size,
                                                             n=n,
                                                             seed_val=seed_val)
    rec_dict['y_classes'] = dfy_train.shape[1]
    rec_dict['x_features'] = dfx_train.shape[1]
    rec_dict['n-train'] = dfx_train.shape[0]
    rec_dict['n-test'] = dfx_test.shape[0]
    rec_dict['seed_val'] = seed_val
    
    # master gurobi
    rec_dict['m_time'] = m_time
    rec_dict['m_console'] = m_console
    
    # master hyper
    rec_dict['alpha'] = alpha
    rec_dict['beta'] = beta
    rec_dict['beta_dict'] = beta_dict
    rec_dict['score_threshold'] = score_threshold
    # rec_dict['alpha'] = 2
    # rec_dict['beta'] = 5
    # rec_dict['score_threshold'] = 1
    
    # sub gurobi
    rec_dict['msub_time'] = 5
    rec_dict['msub_console'] = 0
    
    # adv Train
    rec_dict['c_time'] = c_time # cutting time out | 0: no cutting timeout
    rec_dict['count_miss'] = 0 # include mis classify in ibudget / allowance
    rec_dict['cb'] = 0 # cb on or off
    
    rec_dict['max_itr'] = max_itr # 0: no training | -1 for no iteration limit
    rec_dict['ibudget'] = ibudget # budget per instance
    rec_dict['adv_idx_ls'] = adv_idx_ls
    
    rec_dict['glo_cut_v0'] = glo_cut_v0
    rec_dict['glo_cut_v1'] = glo_cut_v1
    rec_dict['glo_cut_v2'] = glo_cut_v2
    
    rec_dict['wbudget'] = wbudget
    # rec_dict['gbudget'] = gbudget # maximum allowable adv examples + misclass per dataset (only less found will cut)
    # rec_dict['max_itr'] = 10 # 0: no training | -1 for no iteration limit
    # rec_dict['ibudget'] = 2 # budget per instance
    # rec_dict['gbudget'] = 5 # maximum allowable adv examples + misclass per dataset (only less found will cut)
    
    # clear all logs
    clear_log_adv_example_gen()
    clear_log_adv_train()
    clear_log_model_optimize()
    clear_log_adv_cut_v2()
    clear_log_main()
    
    ########################### MASTER ONLY #########################
    
    # build master
    ret_mst_model = model_build(dfx=dfx_train, dfy=dfy_train, 
                                alpha=rec_dict['alpha'], beta=rec_dict['beta'], beta_dict=rec_dict['beta_dict'])    
    
    (m, epos_dict, eneg_dict, wpos_dict, wneg_dict, wnet_dict, bias_dict, bias_bit_dict) = ret_mst_model 
    
    # optimise master
    model_optimize(m, cb=rec_dict['cb'], m_console=rec_dict['m_console'], m_time=rec_dict['m_time'], 
                   threads=rec_dict['threads'])
    
    # save/show accuracy
    print("\n--- Mst Train ---")
    log_main("\n--- Mst Train ---")
    mst_train_acc, mst_test_acc = model_info_acc(bias_dict=bias_dict, wpos_dict=wpos_dict, wneg_dict=wneg_dict, 
                                                 dfx_train=dfx_train, dfy_train=dfy_train, 
                                                 dfx_test=dfx_test, dfy_test=dfy_test)
    rec_dict['mst_train_acc'] = mst_train_acc
    rec_dict['mst_test_acc'] = mst_test_acc
    
    # save model
    record_folder = create_record_folder(dataset=rec_dict['dataset'], 
                                         ibudget=rec_dict['ibudget'],
                                         start_time_main=start_time_main) # have to use to match times w/folder name
    
    record_folder_fullpath = os.path.join(os.getcwd(), record_folder)
    record_folder_uri = pathlib.Path(record_folder_fullpath).as_uri()
    
    if record:
        m.write(record_folder + "master.json")
        m.write(record_folder + "master.lp")
    
    # save base details of designated data points (beta_dict)
    class_ls = [x for x in dfy_train]
    feature_ls = [x for x in dfx_train]
    wcomb_dict = model_wcomb(dfx_train, dfy_train, wpos_dict, wneg_dict, cb=False)
    
    mst_idx_example_ls = {}
    mst_idx_example_count = 0
    for instance_num1, beta_val in beta_dict.items():
        
        x_ins = list(dfx_train.loc[instance_num1])
        x_ins_label = dfy_train.idxmax(axis=1)[instance_num1]
        
        # get instance prediction based on current weights/bias
        x_ins_pred = model_prediction(wcomb_dict, bias_dict, x_ins, class_ls, cb=False, ret='pred')
        x_ins_pred_score = model_prediction(wcomb_dict, bias_dict, x_ins, class_ls, cb=False, ret='score_max')
        x_ins_pred_score_dict = model_prediction(wcomb_dict, bias_dict, x_ins, class_ls, cb=False, ret='score_dict')


        ############### check if adv example can be generated within budget ###############
        
        # check for adv example (status = 1 if found, else 0)
        ret_adv_example_gen = adv_example_gen(dfx_train, dfy_train, # USE instance
                                              wpos_dict, wneg_dict, bias_dict, bias_bit_dict, 
                                              ibudget=rec_dict['ibudget'], adv_idx_ls=[instance_num1],  # PASS INSTANCE IDX
                                              wbudget=rec_dict['wbudget'],
                                              msub_console=rec_dict['msub_console'], msub_time=rec_dict['msub_time'], 
                                              threads=rec_dict['threads'], itr_num=999, 
                                              count_miss=rec_dict['count_miss'], score_threshold=rec_dict['score_threshold'], 
                                              cb=False)
        (adv_status, adv_count, cut_equ_lhs_ls, nval_rhs_ls) = ret_adv_example_gen
        
        print(f"instance_num1: {instance_num1} | label: {x_ins_label} | pred: {x_ins_pred} | correct: {x_ins_label==x_ins_pred} | pred score: {x_ins_pred_score} | gen adv example: {bool(adv_status)} | pred score idx: {x_ins_pred_score_dict}")
        log_main(f"instance_num1: {instance_num1} | label: {x_ins_label} | pred: {x_ins_pred} | correct: {x_ins_label==x_ins_pred} | pred score: {x_ins_pred_score} | gen adv example: {bool(adv_status)} | pred score idx: {x_ins_pred_score_dict}")
        
        if adv_status:
                mst_idx_example_ls[instance_num1] = 1 # record example instances
                mst_idx_example_count = mst_idx_example_count + 1 # record counts

    ########################### ADV TRAIN #########################

    # rebuild master (w/o bias index)
    
    # build master
    ret_mst_model = model_build(dfx=dfx_train, dfy=dfy_train, 
                                alpha=rec_dict['alpha'], beta=rec_dict['beta'], beta_dict=[])    
    
    (m, epos_dict, eneg_dict, wpos_dict, wneg_dict, wnet_dict, bias_dict, bias_bit_dict) = ret_mst_model 
    
    # optimise master
    model_optimize(m, cb=rec_dict['cb'], m_console=rec_dict['m_console'], m_time=rec_dict['m_time'], 
                   threads=rec_dict['threads'])
    
    try:
        # adv training
        ret_adv_model = adv_train(dfx=dfx_train, dfy=dfy_train, 
                                  alpha=rec_dict['alpha'], beta=rec_dict['beta'], 
                                  ibudget=rec_dict['ibudget'], adv_idx_ls=rec_dict['adv_idx_ls'], 
                                  wbudget=rec_dict['wbudget'],
                                  m_time=rec_dict['m_time'], msub_time=rec_dict['msub_time'], 
                                  threads=rec_dict['threads'], max_itr=rec_dict['max_itr'], 
                                  m_console=rec_dict['m_console'], 
                                  msub_console=rec_dict['msub_console'], 
                                  cb=rec_dict['cb'], c_time=rec_dict['c_time'], 
                                  count_miss=rec_dict['count_miss'], 
                                  score_threshold=rec_dict['score_threshold'])

        (m, wpos_dict, wneg_dict, bias_dict, cut_dict) = ret_adv_model
        # rec_dict['cuts'] = len(cut_dict) # causing issues
        
        # save/show accuracy
        print("\n--- Adv Train ---")
        log_main("\n--- Adv Train ---")
        
        adv_train_acc, adv_test_acc = model_info_acc(bias_dict=bias_dict, wpos_dict=wpos_dict, wneg_dict=wneg_dict, 
                                                     dfx_train=dfx_train, dfy_train=dfy_train, 
                                                     dfx_test=dfx_test, dfy_test=dfy_test)
        rec_dict['adv_train_acc'] = adv_train_acc
        rec_dict['adv_test_acc'] = adv_test_acc
        
        rec_dict['status'] = 'completed'
        
        # save details of designated data points (adv_idx_ls) - but adv_example_gen only sees these
        adv_idx_example_ls = {}
        adv_idx_example_count = 0
        for instance_num1, beta_val in beta_dict.items():
            
            x_ins = list(dfx_train.loc[instance_num1])
            x_ins_label = dfy_train.idxmax(axis=1)[instance_num1]
            
            # get instance prediction based on current weights/bias
            x_ins_pred = model_prediction(wcomb_dict, bias_dict, x_ins, class_ls, cb=False, ret='pred')
            x_ins_pred_score = model_prediction(wcomb_dict, bias_dict, x_ins, class_ls, cb=False, ret='score_max')
            x_ins_pred_score_dict = model_prediction(wcomb_dict, bias_dict, x_ins, class_ls, cb=False, ret='score_dict')


            ############### check if adv example can be generated within budget ###############
            
            # check for adv example (status = 1 if found, else 0)
            ret_adv_example_gen = adv_example_gen(dfx_train, dfy_train, # USE instance
                                                wpos_dict, wneg_dict, bias_dict, bias_bit_dict, 
                                                ibudget=rec_dict['ibudget'], adv_idx_ls=[instance_num1],  # PASS INSTANCE IDX
                                                wbudget=rec_dict['wbudget'],
                                                msub_console=rec_dict['msub_console'], msub_time=rec_dict['msub_time'], 
                                                threads=rec_dict['threads'], itr_num=999, 
                                                count_miss=rec_dict['count_miss'], score_threshold=rec_dict['score_threshold'], 
                                                cb=False)
            (adv_status, adv_count, cut_equ_lhs_ls, nval_rhs_ls) = ret_adv_example_gen
            
            print(f"instance_num1: {instance_num1} | label: {x_ins_label} | pred: {x_ins_pred} | correct: {x_ins_label==x_ins_pred} | pred score: {x_ins_pred_score} | gen adv example: {bool(adv_status)} | pred score idx: {x_ins_pred_score_dict}")
            log_main(f"instance_num1: {instance_num1} | label: {x_ins_label} | pred: {x_ins_pred} | correct: {x_ins_label==x_ins_pred} | pred score: {x_ins_pred_score} | gen adv example: {bool(adv_status)} | pred score idx: {x_ins_pred_score_dict}")

            if adv_status:
                adv_idx_example_ls[instance_num1] = 1 # record example instances
                adv_idx_example_count = adv_idx_example_count + 1 # record counts
        

    except KeyboardInterrupt:
        print('interrupted')
        log_main('interrupted')
        rec_dict['status'] = 'interrupted'
    
    # record total for output
    rec_dict['mst_idx_example_ls'] = mst_idx_example_ls
    rec_dict['mst_idx_example_count'] = mst_idx_example_count
    
    rec_dict['adv_idx_example_ls'] = adv_idx_example_ls
    rec_dict['adv_idx_example_count'] = adv_idx_example_count
    
    print(f"mst examples:{mst_idx_example_count} | adv examples:{adv_idx_example_count} | diff:{mst_idx_example_count-adv_idx_example_count}")
    
    # end timer
    end_timer = time.perf_counter()
    rec_dict['time_s'] = f"{end_timer - start_timer:0.0f}"
    
    # save
    if record:
        # save model
        m.write(record_folder + "adv.json")
        m.write(record_folder + "adv.lp")
        rec_dict['record_folder_uri'] = record_folder_uri # add at end
        
        # save logs and experiments
        # shutil.copy2(log_folder + "adv_example_gen.log", record_folder + "adv_example_gen.log")
        # shutil.copy2(log_folder + "adv_train.log", record_folder + "adv_train.log")
        # shutil.copy2(log_folder + "model_optimize.log", record_folder + "model_optimize.log")
        
        # copy and save all logs
        files=os.listdir(log_folder)
        for fname in files:
            shutil.copy2(os.path.join(log_folder,fname), record_folder)
        
        # add record to experiements.csv
        if os.path.isfile(f"records/experiments-{pc_name}.csv"): # append if exists
            with open(f"records/experiments-{pc_name}.csv", 'a', newline='') as f:
                w = csv.DictWriter(f, rec_dict.keys())
                w.writerow(rec_dict)
        else: # otherwise make new 
            with open(f"records/experiments-{pc_name}.csv", 'w', newline='') as f:
                w = csv.DictWriter(f, rec_dict.keys())
                w.writeheader()
                w.writerow(rec_dict)

    # active on Griffon & Henry by default
    pushover_msg = f"""
    ibudget: {rec_dict['ibudget']}
    adv_idx_ls: {rec_dict['adv_idx_ls']}
    
    mst_train_acc: {rec_dict['mst_train_acc']}
    mst_test_acc: {rec_dict['mst_test_acc']}
    
    adv_train_acc: {rec_dict['adv_train_acc']}
    adv_test_acc: {rec_dict['adv_test_acc']}
    
    time_s: {rec_dict['time_s']}
    """
    pushover(title=f"{dataset} | {pc_name} | {rec_dict['ibudget']}", 
             msg=pushover_msg)


In [123]:
data_info()

dataset: zoo             | y_classes: 7     | x_features: 21    | n-train: 70    | n-test: 31   
dataset: student         | y_classes: 8     | x_features: 110   | n-train: 101   | n-test: 44   
dataset: phishing        | y_classes: 3     | x_features: 23    | n-train: 947   | n-test: 406  
dataset: heart           | y_classes: 5     | x_features: 132   | n-train: 212   | n-test: 91   
dataset: wine            | y_classes: 3     | x_features: 150   | n-train: 124   | n-test: 54   
dataset: glass           | y_classes: 6     | x_features: 237   | n-train: 149   | n-test: 65   
dataset: nist            | y_classes: 10    | x_features: 239   | n-train: 2676  | n-test: 1147 
dataset: post-operative  | y_classes: 3     | x_features: 22    | n-train: 63    | n-test: 27   
dataset: flag            | y_classes: 10    | x_features: 287   | n-train: 135   | n-test: 59   
dataset: balance         | y_classes: 3     | x_features: 20    | n-train: 437   | n-test: 188  
dataset: hayes-roth      | y_c

(3, 6, 16, 8)

In [124]:
# main
onedrive('stop') # active on Griffon & Henry by default

######### SELECT PROTECTED DATA POINTS ##########
beta_idx = range(10)
beta_val = [50] * len(beta_idx)
beta_dict = dict(zip(beta_idx, beta_val))  #=> {'a': 1, 'b': 2
adv_idx_ls = [instance_num for instance_num in beta_dict]

dataset = 'post-operative'
# dataset = 'balance'
# dataset = 'phishing'

global glo_adv_ex_dict
global glo_PoolSolutionsN
global glo_cut_v0
global glo_cut_v1
global glo_cut_v2

glo_adv_ex_dict = {} # initialise
glo_cut_v0 = True
glo_cut_v1 = False
glo_cut_v2 = False
glo_PoolSolutionsN = 10

# seed_val = 1 
# seed_val = 12 
# seed_val = 123
seed_val = 1234
# seed_val = 12345
# seed_val = 123456
# seed_val = 1234567
# seed_val = 12345678
# seed_val = 123456789

main(record=0,
     dataset=dataset, 
     n=0,
     max_itr=1000, 
     ibudget=2, 
     adv_idx_ls=adv_idx_ls,
     wbudget=5, 
     alpha=5, 
     beta=10,
     beta_dict=beta_dict,
     score_threshold=1,
     m_time=30,
     m_console=0,
     c_time=0,
     test_size=0.5,
     seed_val=seed_val)

onedrive('start')

print(f"count of adv example: {glo_adv_ex_dict}")
log_main(f"instance/count of adv examples: {glo_adv_ex_dict}")


train/test split w/test_size: 0.5
y_classes: 3 | x_features: 22 | n-train: 45 | n-test: 45
loaded: post-operative
Set parameter Username
Academic license - for non-commercial use only - expires 2023-08-06

--- Mst Train ---
Train Acc: 68.89 | Test Acc: 66.67 | Diff: -2.22
instance_num1: 0 | label: 0 | pred: 0 | correct: True | pred score: 24.0 | gen adv example: False | pred score idx: {0: 24.0, 1: 13.0, 2: 16.0}
instance_num1: 1 | label: 0 | pred: 0 | correct: True | pred score: 25.0 | gen adv example: False | pred score idx: {0: 25.0, 1: 15.0, 2: 15.0}
instance_num1: 2 | label: 0 | pred: 0 | correct: True | pred score: 25.0 | gen adv example: False | pred score idx: {0: 25.0, 1: 14.0, 2: 15.0}
instance_num1: 3 | label: 2 | pred: 0 | correct: False | pred score: 20.0 | gen adv example: False | pred score idx: {0: 20.0, 1: 14.0, 2: 20.0}
instance_num1: 4 | label: 2 | pred: 0 | correct: False | pred score: 20.0 | gen adv example: False | pred score idx: {0: 20.0, 1: 17.0, 2: 20.0}
insta